In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import networkx as nx
import json
import datetime
from random import randrange
from time import sleep
from py2neo import Graph, Node, Relationship
from urllib.parse import unquote
import urllib.request



from TwitterAPI import TwitterAPI, TwitterPager
import datetime
import click
from pathlib import Path

In [2]:
# set path with all files
scrap_path = "C:\\Users\\Jo\\Documents\\Tech\\ScrapData"
path = (scrap_path+"\\20200114_test")
gexfPath = 'C:/Users/Jo/Nextcloud2/Documents/tech/11Medialist/2811NewListD2-IN.gexf'

twitpath = path + "\\twitter"

graph = Graph("bolt://localhost:7687", auth=("neo4j", "Password"))

df_twecoll = pd.read_csv("C:/Users/Jo/Nextcloud2/Documents/tech/twecoll/20200128_Tw_MediaList.dat",header=None)
twecoll_columns =["id","screen_name","relationship","friends","followers","no_se","tweets","start_date","site","picture","location"]
df_twecoll.columns = twecoll_columns


G = nx.read_gexf(gexfPath, node_type=None, relabel=False, version='1.1draft')
data = nx.json_graph.node_link_data(G)

os.chdir(path)
clean_http = re.compile(r"https?://(www\.)?")

ytAPI_client_secrets_file_path = "C:\\Users\\Jo\\Documents\\Tech\\Notebooks\\client_secret_1054515006139-e0p34lt262qbh4af24t2po4c5ophspjo.apps.googleusercontent.com.json"

In [3]:
NoScrapSites = ("discord.gg", "deezer.com", "snapchat.com", "facebook.com", "play.google.com", "youtube.com",
"tumblr.com", "bitly.com", "linkedin.com", "vk.com", "spotify.com", "telegram.org", "apple.co", "netvibes.com",
"google.com", "dailymotion.com", "telegram.me", "amazon.com", "plus.google.com", "mesopinions.com_fr",
"flipboard.com", "tipeee.com", "viadeo.com", "paypal.com", "soundcloud.com", "twitter.com",
"itunes.apple.com", "instagram.com", "wikipedia.org", "pinterest.com", "pearltrees.com", "ausha.co", "jemabonne.fr")

In [4]:
#twitter API :
api_key = "2WIJPjcDeA2VSVYxVfVLnYcVS"
api_secret_key = "bgnd3RKSF3VtqVlk0NX15uhRkPmHKqzLfhQlA3OehCEttXLOeG"


In [5]:
def myUrlParse(url):
    '''parse url and returns correct sitename'''
    from urllib import parse
    # Check liste des exceptions
        
    site_name = parse.urlsplit(url).netloc.replace("www.","").lower()
    
    # Exceptions :
    # 1/ extensions for sites in french
    if parse.urlsplit(url).path.replace("/", "") == "fr" :
        site_name = site_name + "_fr"  
        
    # 2/ same for 20min.ch/ro
    if parse.urlsplit(url).path.replace("/", "") == "ro" :
        site_name = site_name + "_ro"  
        
    # 3/ same for ajplus.net/francais
    if parse.urlsplit(url).path.replace("/", "") == "francais" :
        site_name = site_name + "_francais"    
        
    # 4/ same for gijn.org/gijn-en-francais
    if parse.urlsplit(url).path.replace("/", "") == "gijn-en-francais" :
        site_name = site_name + "_francais"
    
    # 5/ for sites like afrique.tv5monde.com to be reduced to tv5monde.com
    shortersites = ("tv5monde.com","canardpc.com","seloger.com","facebook.com","linkedin.com","pinterest.com",
                    "wikipedia.org","tumblr.com","spotify.com","tendanceouest.com","erfm.fr","viadeo.com",
                    "gulli.fr", "humanite.fr","globalresearch.ca")
    for reduc in shortersites:
        if reduc in site_name:
            site_name = reduc
    
    #6/ Switcher dictionary
    switch_dict = {"amzn.to":"amazon.com", "bit.ly":"bitly.com", "landing.wuaki.tv":"rakuten.tv",
        "t.me":"telegram.org","mrmondialisation.net":"mrmondialisation.org","tv5mondeplus.com":"tv5monde.com", 
                   "fipradio.fr":"fip.fr", "phantasia.be":"mrmondialisation.org", "lesoffrescanal.fr":"canalplus.fr"}
    if site_name in switch_dict:
        site_name = switch_dict[site_name]

        
    # myUrlParse.site_name doit retourner sitename, myUrlParse.url doit retourner l'url nettoyée
    return(site_name)


In [6]:
def getFirstPageNode(url):
    '''From url, download or read firstpage and extract sitename, url and title'''
    site_name = myUrlParse(url)
    
    # exception for noScrapSites list
    if site_name in NoScrapSites:
        return(site_name, "")
    
    title =""
    try:
        # if firstpage file already downloaded
        page = open(site_name + "_FirstPage.html", "r", encoding='utf-8')
        soup = BeautifulSoup(page, "html.parser")
        # print(i, ": opening ", site_name,"from file")
        
    except:
        # if no file, try accessing url
        try:
            page = requests.get(url)
            soup = BeautifulSoup(page.content, "html.parser")
            with open(site_name + "_FirstPage.html", "w", encoding='utf-8') as file:
                file.write(str(soup))
                #print(i, ": downloaded ", site_name, "from site and file written")
            
        except:
            #print("ERROR with ", site_name, ": no file and cannot access site. SKIPPING" )
            error = str(datetime.datetime.now()) + " Error 1 : " + site_name + ": no file and cannot access site. SKIPPING" 
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            
            # Write in dead site file
            with open("DeadSites.txt", "a", encoding='utf-8') as file:
                file.write(site_name + "; " + url +"\n")
            
        
    try:
        title = soup.find("title").get_text()
    except:
        error = str(datetime.datetime.now()) + " Error 2 : cannot parse " + site_name
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")

    return(site_name, title )   


In [7]:
def getAlexaInfoNode(url):
    '''From url or sitename, download or read Alexa page and extract alexa rank'''
    url_alexa_start="https://www.alexa.com/siteinfo/"
    site_name = myUrlParse(url)
    
    # exception for noScrapSites list
    if site_name in NoScrapSites:
        return("")
    
    rank=""
    
    try:
        # if alexa file already downloaded
        page = open(site_name + "_alexa.html", "r", encoding='utf-8')
        soup = BeautifulSoup(page, "html.parser")
        #print(i, ": opening ", site_name,"from file")
        
    except:
        # if no file, try accessing alexa
        try:
            url_alex_full = url_alexa_start + site_name
            page = requests.get(url_alex_full)
            soup = BeautifulSoup(page.content, "html.parser")
        
            with open(site_name + "_alexa.html", "w", encoding='utf-8') as file:
                file.write(str(soup))
            #print(i, ": downloaded ", site_name, "from site and file written")
            
        except:
            error = str(datetime.datetime.now()) + " Error 3 : " + site_name + ": no Alex file and cannot access Alex site. SKIPPING" 
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            #print("ERROR with ", site_name, ": no file and cannot access site. SKIPPING" )
            #err1.append(site_name)
            
       
        sleep_time = randrange(10)
        print("Sleeping for ", sleep_time, "seconds ...")
        sleep(sleep_time)
            
    try:    
        # Rank
        rank = soup.find_all(class_="rankmini-rank")[0].get_text().strip().strip("#").strip(",")
        try:
            rank = int(rank.replace(',', ''))
        except:
            pass

    except:
        error = str(datetime.datetime.now()) + " Error 4 : " + site_name + " no Alexa rank in file" 
        # print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        #print("ERROR parsing Alexa")
        
    return(rank)     

In [8]:
def getHypheInfoNode(url, data = data):
    '''From url or sitename, extract Hyphe info node from gefx file or return site_name if not in gefx'''
    site_name = myUrlParse(url)
    #G = nx.read_gexf(gexfPath, node_type=None, relabel=False, version='1.1draft')
    #data = nx.json_graph.node_link_data(G)
    
    result = site_name
    for n in data['nodes']:
        try:
            if myUrlParse(n['homepage']) == site_name or n['name'].lower() == site_name:
                result = n
        except:
            if n['name'].lower() == site_name:
                result = n
            
    if result == site_name:
        error = str(datetime.datetime.now()) + " Error 5 : " + site_name + " not in hyphe." 
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        with open("NotInHyphe.txt", "a", encoding='utf-8') as file:
            file.write(site_name + "; " + url  + "\n")
                
  
    return(result)

In [9]:
def PushUrlInfoInNeo4j(url):
    title=""
    alex_rank=""
    pages_total=""
    hyphe_date=""
    pages_crawled=""
    label=""
    HypheInfo=""
    site_name = myUrlParse(url)
    
    # exception for noScrapSites list
    if site_name in NoScrapSites:
        a = Node('NoScrap', site_name=site_name)
        a.__primarylabel__ = 'NoScrap'
        a.__primarykey__ = 'site_name'
        graph.merge(a)
        return(a)
    
    if site_name=="":
        error = str(datetime.datetime.now()) + " Error 12 : " + url +" is not a valid URL. Cannot write in db" 
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        return
    
    (site_name, title ) = getFirstPageNode(url)
    alex_rank = getAlexaInfoNode(url)
    HypheInfo = getHypheInfoNode(url)
    if HypheInfo !="":
        
        try:
            pages_total = HypheInfo['pages_total']
            hyphe_date = HypheInfo['last_modification_date']
            pages_crawled = HypheInfo['pages_crawled']
            label = HypheInfo['label']
        except:
            pass
        
    try:    
        a = Node('Website', site_name=site_name, title=title,
             alex_rank = alex_rank, pages_total = pages_total, hyphe_date = hyphe_date,
             pages_crawled = pages_crawled, label = label)
        a.__primarylabel__ = 'Website'
        a.__primarykey__ = 'site_name'
        graph.merge(a)
        return(a)
    except:
        error = str(datetime.datetime.now()) + " Error 6 : Cannot write " + site_name + " in db" 
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
    

In [10]:
def getFacebookUsername(url):
    '''if url is facebook url, returns facebook username. Otherwise returns "" '''
    try:
        res = url.replace('https://www.facebook.com/', '').replace('https://facebook.com/', '').replace('http://www.facebook.com/', '').replace('https://fr-fr.facebook.com/', '').replace('http://fr-fr.facebook.com/','').rstrip('/')
        if res.startswith("pages"):
            m = re.search('/(.+?)/', res)
            if m:
                res = m.group(1)
        if '/' in res:
            res=""
    except:
        res=""
    return(res)

In [11]:
def getTwitterUsername(url):
    '''if url is twitter url, returns twitter username. Otherwise returns "" '''
    try:
        res = url.replace('https://twitter.com/', '').replace('http://twitter.com/', '').replace('?lang=fr', '').replace('?ref_src=twsrc%5Etfw', '').replace('https://www.twitter.com/','').replace('#!/','').rstrip('/')
        res = res.replace(' ','').replace('@','').replace("\n","").replace("http://www.twitter.com/","")
        if '/' in res:
            res=""
        if len(res)>36:
            res=""
        if 'share?' in res:
            res=""
    except:
        res=""
    return(res)

In [12]:
def getYouTubeNameAndurl(url):
    '''extract youtube name (user, chaine...) and start page. If link is channel or no link, returns "" '''
    yt_user=""
    yt_channel=""
    yt_user=""
    try:
        if url.startswith('https://www.youtube.com/user/') | url.startswith('http://www.youtube.com/user/'):
            yt_user = url.replace('https://www.youtube.com/user/','').replace('http://www.youtube.com/user/','').rstrip('/').lower()
        if url.startswith('https://www.youtube.com/channel/') | url.startswith('https://youtube.com/channel/'):
            yt_user = "channel"
        if '/' not in url.replace("https://www.youtube.com/c/","").replace('http://www.youtube.com/','').replace('https://www.youtube.com/',''):
            yt_user = url.replace("https://www.youtube.com/c/","").replace('http://www.youtube.com/','').replace('https://www.youtube.com/','').lower()
        if yt_user=="":
            url=""
    except:
        url=""
    return(yt_user,url)

In [13]:
def getFirstPageRSNode(url):
    '''From url or sitename, read firstpage and extract Social Network info'''
    site_name = myUrlParse(url)
    (fb,twit,yt_user,yt_url,sc,insta,dm)=("","","","","","","")
    
    # exception for noScrapSites list
    if site_name in NoScrapSites:
        return(fb,twit,yt_user,yt_url,sc,insta,dm)
        
    try:
        # if firstpage file already downloaded
        page = open(site_name + "_FirstPage.html", "r", encoding='utf-8')
        soup = BeautifulSoup(page, "html.parser")
        # print(i, ": opening ", site_name,"from file")
        
    except:
        # if no file, error
        error = str(datetime.datetime.now()) + " Error 7 - " + site_name + ": no firstpage file. Please download first " 
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
    
    try:
        for link in soup.find_all('a'):
            link_url=str(link.get('href'))
            if "facebook" in link_url:
                fb = getFacebookUsername(link_url).lower()            
            if "twitter" in link_url:
                twit = getTwitterUsername(link_url).lower()
            if "youtube" in link_url:
                (yt_user,yt_url) = getYouTubeNameAndurl(link_url)
                if yt_user == "channel":
                    yt_user = site_name
# Removed snapchat, insta and dailymotion.    
# TODO : build SC, Insta & Dailymotion url parser
            '''if "snapchat" in link_url:
                sc = link_url
            if "instagram" in link_url:
                insta = link_url
            if "dailymotion" in link_url:
                dm = link_url'''
    except:
        error = str(datetime.datetime.now()) + " Error 8 : " + site_name + ": cannot parse RS" 
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
            
                
    return(fb,twit,yt_user,yt_url,sc,insta,dm)

In [14]:
def PushRSUrlInfoInNeo4j(url):
    '''From url or sitename, read firstpage, extract SN info and push nodes and relationship in neo4j'''
    
    (site_name, title ) = getFirstPageNode(url)
    siteNodeRes = graph.nodes.match("Website", site_name=site_name)
    sitenodenb = len(siteNodeRes)
    
    if sitenodenb != 1:
        error = str(datetime.datetime.now()) + " Error 9 : " + site_name + " has " + str(sitenodenb) + " nodes in DB"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
    else:
        siteNode=siteNodeRes.first()
        (fb,twit,yt_user,yt_url,sc,insta,dm)= getFirstPageRSNode(url)
     
        tx = graph.begin()

        if fb != "":
            fb_n = Node("Facebook", user = fb)
            tx.merge(fb_n, "Facebook", "user" )
            site_fb_r = Relationship(siteNode, "LINKS_TO", fb_n)
            tx.merge(site_fb_r)
        
        if twit != "":
            twit_n = Node("Twitter", user = twit)
            tx.merge(twit_n, "Twitter", "user" )
            site_twit_r = Relationship(siteNode, "LINKS_TO", twit_n)
            tx.merge(site_twit_r)
    
        if yt_user != "":
            yt_n = Node("Youtube", user = yt_user, url = yt_url )
            tx.merge(yt_n, "Youtube", "user" )
            site_yt_r = Relationship(siteNode, "LINKS_TO", yt_n)
            tx.merge(site_yt_r)
    
        tx.commit()
    return(siteNode)


In [15]:
def PushHypheLinksToNeo4j(url):
    '''From url or sitename, extract hyphe connections and push them in Neo4j if nodes exists'''
    ''' Gives back connections to and from this site that does not exist in DB'''
    site_name = myUrlParse(url)
    source_match = ''
    sourceID = ''
    
    # exception for noScrapSites list
    if site_name in NoScrapSites:
        return
    
    # print("1 start find site hyphe ID")
    
    # Find site hyphe ID
    for n in data['nodes']:
        if n['name'].lower() == site_name:
            sourceID = n['id']
            source_match = graph.nodes.match("Website", site_name = site_name)
            if len(source_match) != 1:
                error = str(datetime.datetime.now()) + " Error 10 : " + site_name + " has " + str(len(source_match)) + " nodes in DB"
                print(error)
                with open("error_file.txt", "a", encoding='utf-8') as file:
                    file.write(error +"\n")
    
    # print("2 sourceID = ", sourceID)
    
    if (sourceID == ''):
        error = str(datetime.datetime.now()) + " Error 11 : " + site_name + " has no connection in Hyphe."
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
      
    else:
        # print("3 start links loop")
        
        # Find links where ID is source
        for l in data['links']:
            if l['source']== str(sourceID):
                
                # print("4 : l = ", l)
                
                for n in data['nodes']:
                    # find target
                    if n['id']==str(l['target']):
                        target_name = n['name'].lower()
                        # print(target_name)
                        target_match = graph.nodes.match("Website", site_name = target_name)
                        # print(len(target_match))
                        
                        if (len(target_match) != 1):
                            error = str(datetime.datetime.now()) + " Error 12 : " + target_name + " has " + str(len(target_match)) + " nodes in DB"
                            print(error)
                            with open("error_file.txt", "a", encoding='utf-8') as file:
                                file.write(error +"\n")
                        
                        else:
                            source_n = source_match.first()
                            target_n = target_match.first()
                            # print(source_n, target_n)
                        
                        
                        # create connections
                        if ((len(source_match) != 1) or (len(target_match) != 1)):
                            # print("lensource = ", len(source_match))
                            # print("lentarget = ",len(target_match))
                            error = str(datetime.datetime.now()) + " Error 13 : " + l['id'] + " cannot be created in db"
                            print(error)
                            with open("error_file.txt", "a", encoding='utf-8') as file:
                                file.write(error +"\n")
                
                        else:
                            # print("start creating relationship in neo4j")
                            tx = graph.begin()
                            source_n = source_match.first()
                            target_n = target_match.first()
                            rel = Relationship(source_n, "LINKS_TO", target_n)
                            rel["count_D2"]=l['count']
                            tx.merge(rel)
                            tx.commit()   
                            #print("Created ", source_n['label'], " - ", target_n['label'], "connection from hyphe" ) 
                        


# TODO : 
- [] check errors
- [] actualise twitter info (date, lieu, nom... Trouver comment avoir site !! )
- [] Que faire quand pas fichier dat ou twt (pour faire au cas par cas)
- [] check twitter id dans base. SI yapas, check dans fichier

In [16]:
def ImportTwitterFollowInNeo4j(url):
    site_name = myUrlParse(url)
    twitter_n = ""
      
    # Trouve si url est dans la base et si point vers un twitter account    
    for r in graph.match((graph.nodes.match(site_name=site_name).first(),)):
        if r.end_node.has_label("Twitter"):
            twitter_n = r.end_node
            
    if twitter_n == "":
        error = str(datetime.datetime.now()) + " Error 14 : " + str(url) + " has no twitter node in db"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        return
    
    # If no twitter ID is base, find it in file
            
    if twitter_n['tw_id'] == None:
        try:
            # Write ID in neo4j
            twitter_n['tw_id'] = int(df_twecoll.loc[df_twecoll['screen_name']==twitter_n['user'], 'id'].iloc[0])
        except:
            error = str(datetime.datetime.now()) + " Error 16 : " + twitter_n['user'] + " has no twecoll info"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return

    # Open twecoll file corresponding to ID
    file_id = "C:/Users/Jo/Nextcloud2/Documents/tech/twecoll/fdat/" + str(twitter_n['tw_id']) + ".f"
    
    try:
        with open(file_id) as f:
            content = f.readlines()
            # remove whitespace characters like `\n` at the end of each line
            content = [x.strip() for x in content]
            
            for c in content:
                if int(c) in list(df_twecoll['id']):
                    follows_username = df_twecoll.loc[df_twecoll['id']==int(c), 'screen_name'].iloc[0].lower()
                    target_n = graph.nodes.match("Twitter",user = follows_username).first()
                    if target_n == None:
                        error = str(datetime.datetime.now()) + " Error 15 : " + follows_username + " has no twitter node in db"
                        print(error)
                        with open("error_file.txt", "a", encoding='utf-8') as file:
                            file.write(error +"\n")
                        continue
                    target_n["tw_id"] = c
            
                    tx = graph.begin()
                    rel = Relationship(twitter_n, "FOLLOWS", target_n)
                    tx.push(target_n)
                    tx.push(twitter_n)
                    tx.merge(rel)
                    tx.commit()
                    #print(rel)
    
    
    
    except:
        error = str(datetime.datetime.now()) + " Error 17 : " + twitter_n['user'] + " has no twecoll info"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        with open("NoTwecoll.txt", "a", encoding='utf-8') as file:
            file.write(twitter_n['user'] +"\n")
        return
    
        
        

# Twitter API

In [17]:
def create_api():
    config = dict(
        twitter=dict(
            api_key=api_key,
            api_secret_key=api_secret_key
        )
    )
    api = TwitterAPI(config['twitter']['api_key'],
                     config['twitter']['api_secret_key'],
                     auth_type='oAuth2'
                     )
    return (api)


In [18]:
api=create_api()

In [19]:
def respectful_api_request(*args):
    '''Respects api limits and retries after waiting.'''
    r = api.request(*args)
    if r.headers['x-rate-limit-remaining'] == '0':
        waiting_time = int(
            r.headers['x-rate-limit-reset']) - int(round(time.time()))
        click.echo(
            'Hit the API limit. Waiting for refresh at {}.'
            .format(datetime.datetime.utcfromtimestamp(int(r.headers['x-rate-limit-reset']))
                    .strftime('%Y-%m-%dT%H:%M:%SZ')))
        time.sleep(waiting_time)
        return (respectful_api_request(*args))
    return(r)

In [20]:
def write_twitpro_file(screen_name):
    '''
    Returns twitter profile file path from twitter user name
    if no file, download it using twitter API
    '''
    screen_name = screen_name.lower()
    # If no file, download :
    cursor = -1
    twitpro_file = path + "/twitter/" + screen_name + '.twitprofile'
    
    if not Path(twitpro_file).is_file():
        try:
            r = respectful_api_request('users/show', {'screen_name': screen_name, 'cursor': cursor})
            for items in r:
                with open(twitpro_file, "w", encoding='utf-8') as file:
                    file.write(str(items))
            return(twitpro_file)
            
        except:
            error = str(datetime.datetime.now()) + " Error 18 : " + screen_name + " is not a twitter user"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return("")
    else:
        return(twitpro_file)          

In [21]:
def push_twitpro_in_Neo4j(screen_name):
    '''
    From twitter screen name, download twitter profile info and push into Neo4j. 
    Save intermediary twitpro file if it does not exists
    '''
    screen_name = screen_name.lower()
    keys = ['id','name','screen_name','location','description','desc_url','url','followers_count',
                  'friends_count', 'listed_count', 'created_at', 'verified', 'statuses_count']
    
    twitpro_file = write_twitpro_file(screen_name)
    
    if twitpro_file != "":
    # if file exists, reads it and parse into twit_profile_extracted_keys
        with open(twitpro_file, encoding="utf8") as f:
            twit_profile = eval(f.read())
            twit_profile_extracted_keys = {k: twit_profile.get(k) for k in keys}
            if twit_profile.get('entities').get('url')==None:
                twit_profile_extracted_keys['url']= None
            else:
                twit_profile_extracted_keys['url']= twit_profile.get('entities')['url']['urls'][0]['expanded_url']
    
       # Finds twitter node
        twitter_n = graph.nodes.match("Twitter", user = screen_name).first()
        # update properties
        for k in keys:
            twitter_n[k] = twit_profile_extracted_keys[k]
        # push in DB
        tx = graph.begin()
        tx.push(twitter_n)
        tx.commit()
        return(twitter_n)
    else:
        return("")


# Youtube API

In [22]:
def create_YT_API():
    
    import os
    import google_auth_oauthlib.flow
    import googleapiclient.discovery
    import googleapiclient.errors
    import json
    
    scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = ytAPI_client_secrets_file_path
    
    flow = google_auth_oauthlib.flow.InstalledAppFlow.from_client_secrets_file(client_secrets_file, scopes)
    credentials = flow.run_console()
    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, credentials=credentials)
    
    return(youtube)


In [23]:
youtube = create_YT_API()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1054515006139-e0p34lt262qbh4af24t2po4c5ophspjo.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyoutube.readonly&state=CwElZGFRzdklbHFaQD7P8s8gN5eRjr&prompt=consent&access_type=offline
Enter the authorization code: 4/yAE6twKL7MB8_KpNSPSjaJtZwejBq9IE_Py6l6EEFmwDAF9KMSVPJW4


In [24]:
def write_ytpro_file(screen_name):
    ''' If file doesnt exists, from youtube screen name download profile info and write on file
    returns file location or "" if user does not exists   
    '''
    
    screen_name = screen_name.lower()
    ytpro_file = path + "/youtube/" + screen_name + '.ytprofile'
    
    if not Path(ytpro_file).is_file():
        try:
            request = youtube.channels().list(
                part="brandingSettings,contentDetails,id,snippet,statistics",
                forUsername= screen_name
            )
            response = request.execute()
            
            if len(response["items"])==0:
                error = str(datetime.datetime.now()) + " Error 19 : " + screen_name + " is not a youtube user"
                print(error)
                with open("error_file.txt", "a", encoding='utf-8') as file:
                    file.write(error +"\n")
                return("")
                                             
            with open(ytpro_file, "w", encoding='utf-8') as file:
                file.write(str(response['items'][0]))
                                
            return(ytpro_file)
        
            
        except:
            error = str(datetime.datetime.now()) + " Error 19 : " + screen_name + " is not a youtube user"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return("")
        
    else:
        return(ytpro_file)    
    

In [25]:
def write_ytsub_file(screen_name):
    ''' If file doesnt exists, from youtube screen name download public subscriptions  and write on file
    returns file location or "" if user does not exists or subscriptions not public  
    '''
    
    screen_name = screen_name.lower()
    ytpro_file = write_ytpro_file(screen_name)
    ytsub_file = path + "/youtube/" + screen_name + '.ytsubscriptions'
    
    if ytpro_file != "":
    # if file exists, reads it and parse into twit_profile_extracted_keys
        with open(ytpro_file, encoding="utf8") as f:
            yt_profile = eval(f.read())
        
        yt_channelID = yt_profile['id']
        
        subscriptions =[]
        
        if not Path(ytsub_file).is_file():
                    
            try: #call for public subscriptions
                request = youtube.subscriptions().list(
                    part="snippet,contentDetails",
                    channelId=yt_channelID,
                    maxResults=50
                )
                res = request.execute()
            
                for item in res['items']:
                    subscriptions.append(item)
            
                try:
                    nextPageToken = res['nextPageToken']
            
                except:
                    nextPageToken = None
            
                while (nextPageToken):
                    try:
                        request = youtube.subscriptions().list(
                            part="snippet,contentDetails",
                            channelId=yt_channelID,
                            maxResults=50,
                            pageToken=nextPageToken
                        )
                        res = request.execute()


                        for item in res['items']:
                            subscriptions.append(item)

                        nextPageToken = res['nextPageToken']

                    except:
                        break
                
                with open(ytsub_file, "w", encoding='utf-8') as file:
                    file.write(str(subscriptions))
            
                return(ytsub_file)
                
            except:
                ytsub_file=""
                return(ytsub_file)
            
        else:
            return(ytsub_file)
        
    else:
        return(ytsub_file)

In [26]:
def write_ytvideos_file(screen_name):
    ''' If file doesnt exists, from youtube screen name download videos info from upload playlist and write on file
    returns file location or "" if user does not exists or no upload playlist  
    '''
    
    screen_name = screen_name.lower()
    ytpro_file = write_ytpro_file(screen_name)
    ytvideo_file = path + "/youtube/" + screen_name + '.ytvideos'
    
    if ytpro_file != "":
        with open(ytpro_file, encoding="utf8") as f:
            yt_profile = eval(f.read())
            
        upload_playlist_ID = yt_profile['contentDetails']['relatedPlaylists']['uploads']
        videos =[]
        
        if not Path(ytvideo_file).is_file():

            try:
                request = youtube.playlistItems().list(
                    part="snippet,contentDetails",
                    maxResults=50,
                    playlistId=upload_playlist_ID
                    )
                response_playlistItems = request.execute()
            
                for item in response_playlistItems['items']:
                    videos.append(item)
            
                try:
                    nextPageToken = response_playlistItems['nextPageToken']
            
                except:
                    nextPageToken = None
            
                while (nextPageToken):
                    try:
                        request = youtube.playlistItems().list(
                        part="snippet,contentDetails",
                        maxResults=50,
                        playlistId=upload_playlist_ID,
                        pageToken=nextPageToken        
                        )
                        res = request.execute()
        
                        for item in res['items']:
                            videos.append(item)
            
                        nextPageToken = res['nextPageToken']

                    except:
                        break
            
                with open(ytvideo_file, "w", encoding='utf-8') as file:
                    file.write(str(videos))
            
                return(ytvideo_file)
            
            except:
                ytvideo_file =""
                return(ytvideo_file)
            
        else:
            return(ytvideo_file)
                  
    else:
        return("")

In [27]:
def write_ytpro_file_fromChannelID(channelID):
    ''' If file doesnt exists, from youtube screen name download profile info and write on file
    returns file location or "" if user does not exists   
    '''
    
    # Check if channelID already in base, in this case return yt_profile from base
    for nodes in graph.nodes.match("Youtube"):
        if nodes['channelID'] == channelID:
            if nodes['ytpro_file']:
                ytpro_file=nodes['ytpro_file']
            else:
                if nodes['customUrl']:
                    screen_name = nodes['customUrl']
                else:
                    if nodes['title']:
                        screen_name = nodes['title'].lower().replace(" ", "_")
                    else:
                        print("this channelID has no title!!!!")
                        return("")
                screen_name = screen_name.lower()
                ytpro_file = path + "/youtube/" + screen_name + '.ytprofile'
                
            return(ytpro_file)
    
    request = youtube.channels().list(
        part="brandingSettings,contentDetails,id,snippet,statistics",
        id= channelID
        )
    response = request.execute()
    
    if len(response["items"])==0:
        error = str(datetime.datetime.now()) + " Error 20 : " + channelID + " is not a youtube channelID"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        return("")
    
    try:
        screen_name = response["items"][0]['snippet']['customUrl']
    
    except:
        try:
            screen_name = response['items'][0]['snippet']['title'].lower().replace(" ", "_")
        except:
            print("this channelID has no title!!!!")
            return("")
    
    screen_name = screen_name.lower()
    ytpro_file = path + "/youtube/" + screen_name + '.ytprofile'
        
    if not Path(ytpro_file).is_file():
        try:
            with open(ytpro_file, "w", encoding='utf-8') as file:
                file.write(str(response['items'][0]))
                                
            return(ytpro_file)
        except:
            print(screen_name, " is to weird for us. CHange the name !")
            return("")
                
    else:
        return(ytpro_file)   

In [28]:
def write_yt_links_file(screen_name):
    '''   
    '''
    
    screen_name = screen_name.lower()
    ytpro_file = write_ytpro_file(screen_name)
    ytlinks_file = path + "/youtube/" + screen_name + '.ytlinks'
    
    if Path(ytlinks_file).is_file():
        return(ytlinks_file)
    
    if ytpro_file =="":
        return("")
    
    with open(ytpro_file, encoding="utf8") as f:
        yt_profile = eval(f.read())
    
    about_url = "https://www.youtube.com/channel/" + yt_profile['id'] +"/about"
    about_page = urllib.request.urlopen(about_url)
    about_soup = BeautifulSoup(about_page)
    links =[]
    
    for a in about_soup.find_all("a", class_ = "about-channel-link"):
        for ref in a['href'].split('q='):
            if 'http' in ref:
                about_title = a['title']
                about_link = unquote(ref.split('&')[0]).split('#utm')[0]
                already=False
                for link in links:
                    if link['url']== about_link:
                        already=True
                if not already:
                    links.append({'title':about_title,'url':about_link})
                    
    with open(ytlinks_file, "w", encoding='utf-8') as file:
                file.write(str(links))
    
            
    return(ytlinks_file)


In [29]:
def write_yt_files_from_yt_url(url):
    screen_name=""
    
    CustomChannels = {"alimentationgenerale1":"UCgmfhT7laKrH4C4d3KnqyNw","numerama":"UCAz-755tH3m8_BwaluzdJwQ",
    "FondationPol%C3%A9mia":"UCMf8__hXS4X7j2eTY37Mqiw","mercato":"UCrzDtXyuSBch2u_31JJj-Dw",
    "SanteplusmagOfficiel":"UC_c2DThPTzuw3DB3fdNbZAw",
    "TouteleuropeEu":"UCvrqvZ1ABclMKlMLQKfutQA","TSAToutsurlAlg%C3%A9rie":"UC0a2EoPfE6Rh50V8ZDqeH0w",
    "ajplussaha":"UCEg4_mU2CqqtQxWwzL_uh-w","aleteiafr":"UCphfPDeHIsRXcQ_v7nIjL1Q",
    "letemps":"UCam7yYDSKFIpwAscmyzOsQQ","euractiv":"UCDCgvwyjJqZWYs6j0D2FF6g"}
    
    # if url is not 
    if myUrlParse(url) != "youtube.com":
        error = str(datetime.datetime.now()) + " Error 21 : " + url + " is not a youtube url"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        return    
    
    
    # check if channelID
    if "youtube.com/channel" in url:
        channelID = url.replace('https://www.youtube.com/channel/',"").replace('https://youtube.com/channel/',"").replace("/featured","")[0:24]
        channelID = channelID.split("/")[0]
        ytpro_file = write_ytpro_file_fromChannelID(channelID)
          
        try:
            with open(ytpro_file, encoding="utf8") as f:
                yt_profile = eval(f.read())
    
            try:
                screen_name= yt_profile['snippet']['customUrl']

            except:
                screen_name = yt_profile['snippet']['title'].lower().replace(" ", "_")

        except:
            error = str(datetime.datetime.now()) + " Error 23 : "+ channelID + " is not a valid youtube channel"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return            
        
    
    # check if user
    if "user" in url:
        url=url.replace("?feature=watch", "").replace("/videos","").replace('?sub_confirmation=1','').replace("?feature=guide","")
        screen_name = url.split("user/")[1].split("/")[0]
        ytpro_file = write_ytpro_file(screen_name)
        
    
    # This code doesn't take any other format
    # check if results
    if "youtube.com/results" in url:
        if url == "https://www.youtube.com/results?search_query=riposte+laique":
            channelID = "UCv7I-SNC4Con0BfdLEWHUPA"
            ytpro_file = write_ytpro_file_fromChannelID(channelID)
            screen_name = "ciceropicas"
        else:
            return("results")
    
    # Check if playlist
    if "youtube.com/playlist" in url:
        return("playlist")
    
    # else if link du type http://www.youtube.com/watch?v=dtUH2YSFlVU, fait rien    
    if "youtube.com/watch" in url:
        return("watch")
    
    
    # If url is like that : "https://www.youtube.com/c/alimentationgenerale1" or "http://www.youtube.com/taranisnews"
    if screen_name=="":
        if "/c/" in url:
            customName = url.split("/c/")[1]
        else:
            customName = url.split("youtube.com/")[1]
        
        if customName in CustomChannels:
            channelID = CustomChannels[customName]
        
        else:
            # ID request
            request = youtube.channels().list(
                part="id",
                forUsername = customName
                )
            response = request.execute()
            try:
                channelID = response['items'][0]['id']
            except:
                error = str(datetime.datetime.now()) + " Error 24 : "+ url + " is not a valid youtube url"
                print(error)
                with open("error_file.txt", "a", encoding='utf-8') as file:
                    file.write(error +"\n")
                return
        
        ytpro_file = write_ytpro_file_fromChannelID(channelID)
        
        try:
            with open(ytpro_file, encoding="utf8") as f:
                yt_profile = eval(f.read())
    
            try:
                screen_name= yt_profile['snippet']['customUrl']

            except:
                screen_name = yt_profile['snippet']['title'].lower().replace(" ", "_")

        except:
            error = str(datetime.datetime.now()) + " Error 25 : "+ channelID + " is not a valid youtube channel"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return   
    
    
    # check if customURL cohérent
           
    ytsub_file = write_ytsub_file(screen_name)
    
    try:
        with open(ytsub_file, encoding="utf8") as f:
            yt_sub = eval(f.read())
    except:
        yt_sub =""
            
    yt_links_file = write_yt_links_file(screen_name)
        
    return(ytpro_file,ytsub_file,yt_links_file)
   

In [30]:
def write_yt_pro_file_from_yt_url(url):
    screen_name=""
    
    CustomChannels = {"alimentationgenerale1":"UCgmfhT7laKrH4C4d3KnqyNw","numerama":"UCAz-755tH3m8_BwaluzdJwQ",
    "FondationPol%C3%A9mia":"UCMf8__hXS4X7j2eTY37Mqiw","mercato":"UCrzDtXyuSBch2u_31JJj-Dw",
    "SanteplusmagOfficiel":"UC_c2DThPTzuw3DB3fdNbZAw",
    "TouteleuropeEu":"UCvrqvZ1ABclMKlMLQKfutQA","TSAToutsurlAlg%C3%A9rie":"UC0a2EoPfE6Rh50V8ZDqeH0w",
    "ajplussaha":"UCEg4_mU2CqqtQxWwzL_uh-w","aleteiafr":"UCphfPDeHIsRXcQ_v7nIjL1Q",
    "letemps":"UCam7yYDSKFIpwAscmyzOsQQ","euractiv":"UCDCgvwyjJqZWYs6j0D2FF6g"}
    
    # if url is not 
    if myUrlParse(url) != "youtube.com":
        error = str(datetime.datetime.now()) + " Error 21 : " + url + " is not a youtube url"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        return    
    
    
    # check if channelID
    if "youtube.com/channel" in url:
        channelID = url.replace('https://www.youtube.com/channel/',"").replace('https://youtube.com/channel/',"").replace("/featured","")[0:24]
        channelID = channelID.split("/")[0]
        ytpro_file = write_ytpro_file_fromChannelID(channelID)
          
        try:
            with open(ytpro_file, encoding="utf8") as f:
                yt_profile = eval(f.read())
    
            try:
                screen_name= yt_profile['snippet']['customUrl']

            except:
                screen_name = yt_profile['snippet']['title'].lower().replace(" ", "_")

        except:
            error = str(datetime.datetime.now()) + " Error 23 : "+ channelID + " is not a valid youtube channel"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return            
        
    
    # check if user
    if "user" in url:
        url=url.replace("?feature=watch", "").replace("/videos","").replace('?sub_confirmation=1','').replace("?feature=guide","")
        screen_name = url.split("user/")[1].split("/")[0]
        ytpro_file = write_ytpro_file(screen_name)
        
    
    # This code doesn't take any other format
    # check if results
    if "youtube.com/results" in url:
        if url == "https://www.youtube.com/results?search_query=riposte+laique":
            channelID = "UCv7I-SNC4Con0BfdLEWHUPA"
            ytpro_file = write_ytpro_file_fromChannelID(channelID)
            screen_name = "ciceropicas"
        else:
            return("results")
    
    # Check if playlist
    if "youtube.com/playlist" in url:
        return("playlist")
    
    # else if link du type http://www.youtube.com/watch?v=dtUH2YSFlVU, fait rien    
    if "youtube.com/watch" in url:
        return("watch")
    
    
    # If url is like that : "https://www.youtube.com/c/alimentationgenerale1" or "http://www.youtube.com/taranisnews"
    if screen_name=="":
        if "/c/" in url:
            customName = url.split("/c/")[1]
        else:
            customName = url.split("youtube.com/")[1]
        
        if customName in CustomChannels:
            channelID = CustomChannels[customName]
        
        else:
            # ID request
            request = youtube.channels().list(
                part="id",
                forUsername = customName
                )
            response = request.execute()
            try:
                channelID = response['items'][0]['id']
            except:
                error = str(datetime.datetime.now()) + " Error 24 : "+ url + " is not a valid youtube url"
                print(error)
                with open("error_file.txt", "a", encoding='utf-8') as file:
                    file.write(error +"\n")
                return
        
        ytpro_file = write_ytpro_file_fromChannelID(channelID)
        
        try:
            with open(ytpro_file, encoding="utf8") as f:
                yt_profile = eval(f.read())
    
            try:
                screen_name= yt_profile['snippet']['customUrl']

            except:
                screen_name = yt_profile['snippet']['title'].lower().replace(" ", "_")

        except:
            error = str(datetime.datetime.now()) + " Error 25 : "+ channelID + " is not a valid youtube channel"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return   
    
    
    return(ytpro_file)

In [31]:
def push_yt_profile_inNeo4j(url):
    '''
    For existing yt node : from its url download the yt pro files and push youtube node with profile data
    returns node or if node doesn't exist, do nothing    
    '''
        
    keys = {'id':'channelID', 
        'snippet/title':'title', 
        'snippet/description':'description',
        'snippet/customUrl':'customUrl', 
        'snippet/publishedAt':'publishedAt',
        'snippet/country':'country', 
        'contentDetails/relatedPlaylists/uploads':'uploadPlaylist',
        'statistics/viewCount':'viewCount', 
        'statistics/commentCount':'commentCount',
        'statistics/subscriberCount':'subscriberCount', 
        'statistics/hiddenSubscriberCount':'hiddenSubscriberCount',
        'statistics/videoCount':'videoCount',
        'brandingSettings/channel/keywords':'keywords'
       }
    
    # find node with the corresponding youtube url
    yt_n = graph.nodes.match(url=url).first()
    if yt_n == None:
        return
    
    # download les files
    if write_yt_files_from_yt_url(url)in (None, "watch" , "playlist" , "results"):
        if write_yt_files_from_yt_url(url)== None:
            errortype =""
        else:
            errortype=write_yt_files_from_yt_url(url)
        error = str(datetime.datetime.now()) + " Error 22 : "+ errortype +" - "+ url + " is not a valid youtube url. Please check the corresponding yt node"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
        return 
        
    ytpro_file = write_yt_pro_file_from_yt_url(url)
    
    # update node info with youtube profile info
    with open(ytpro_file, encoding="utf8") as f:
        yt_profile = eval(f.read())
    
    yt_n['ytpro_file'] = ytpro_file
    
    for key in keys:
        try:
            if len(key.split('/'))==1:
                yt_n[keys[key]]= yt_profile[key]
            if len(key.split('/'))==2:
                yt_n[keys[key]] = yt_profile[key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
            if len(key.split('/'))==3:
                yt_n[keys[key]] = yt_profile[key.split('/')[len(key.split('/'))-3]][key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
        except:
            pass
            
    # push le node
    tx = graph.begin()
    tx.push(yt_n)
    tx.commit()
    return(yt_n)

In [32]:
# not used... Delete ?

def push_yt_featured_channels_inNeo4j(url):
    '''
    '''
    keys = {'id':'channelID', 
        'snippet/title':'title', 
        'snippet/description':'description',
        'snippet/customUrl':'customUrl', 
        'snippet/publishedAt':'publishedAt',
        'snippet/country':'country', 
        'contentDetails/relatedPlaylists/uploads':'uploadPlaylist',
        'statistics/viewCount':'viewCount', 
        'statistics/commentCount':'commentCount',
        'statistics/subscriberCount':'subscriberCount', 
        'statistics/hiddenSubscriberCount':'hiddenSubscriberCount',
        'statistics/videoCount':'videoCount',
        'brandingSettings/channel/keywords':'keywords'
       }
    
    # find node with the corresponding youtube url
    yt_n = graph.nodes.match(url=url).first()

    # check les files
    (ytpro_file,ytsub_file,yt_links_file) = write_yt_files_from_yt_url(url)

    with open(ytpro_file, encoding="utf8") as f:
        yt_profile = eval(f.read())
    
    try: 
        for channel in yt_profile['brandingSettings']['channel']['featuredChannelsUrls']:
            print(channel)   
            #cherche si ya dejà un node avec channelID
            yt_featchan_n = graph.nodes.match(channelID=channel).first()
        
            feat_chan_file = write_ytpro_file_fromChannelID(channel)
            with open(feat_chan_file, encoding="utf8") as file:
                feat_chan_pro = eval(file.read())
            
            if not yt_featchan_n:
                # créé node from pro_fil
                # créer node yt_featchan_n    
                yt_user = feat_chan_file.replace(path + "/youtube/","").replace('.ytprofile',"")
                yt_featchan_n = Node("Youtube", user = yt_user)
    
            
            # fill node properties from pro_file:
            for key in keys:
                try:
                    if len(key.split('/'))==1:
                        yt_featchan_n[keys[key]]= feat_chan_pro[key]
                    if len(key.split('/'))==2:
                        yt_featchan_n[keys[key]] = feat_chan_pro[key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
                    if len(key.split('/'))==3:
                        yt_featchan_n[keys[key]] = feat_chan_pro[key.split('/')[len(key.split('/'))-3]][key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
                except:
                    pass
    
        
            source_n = yt_n
            target_n = yt_featchan_n
            rel = Relationship(source_n, "LINKS_TO", target_n)
            rel["yt_type"]="featured_Channel"
            #tx = graph.begin()
            #tx.merge(yt_featchan_n, "Youtube", "user" )
            #tx.merge(rel)
            #tx.commit() 
            print("source: ", source_n['user']," target: ",target_n['user'], " rel:", rel)

    except:
        pass
    

In [32]:
def push_ytnode_featured_channels_inNeo4j(yt_n):
    '''
    from yt node, create featured channels nodes and LINKS_TO links
    returns all relationships as list of relationships 
    '''
    
    keys = {'id':'channelID', 
        'snippet/title':'title', 
        'snippet/description':'description',
        'snippet/customUrl':'customUrl', 
        'snippet/publishedAt':'publishedAt',
        'snippet/country':'country', 
        'contentDetails/relatedPlaylists/uploads':'uploadPlaylist',
        'statistics/viewCount':'viewCount', 
        'statistics/commentCount':'commentCount',
        'statistics/subscriberCount':'subscriberCount', 
        'statistics/hiddenSubscriberCount':'hiddenSubscriberCount',
        'statistics/videoCount':'videoCount',
        'brandingSettings/channel/keywords':'keywords'
       }
    all_rel =[]
    # from node, go to pro_file 
    if yt_n['ytpro_file']:
        ytpro_file = yt_n['ytpro_file']
    else:
        try:
            ytpro_file = push_yt_profile_inNeo4j(yt_n['url'])['ytpro_file']
        except:
            error = str(datetime.datetime.now()) + " Error 23b : node" +str(yt_n) + " has no valid profile"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return
    
   


    # check si node exists (from channelID)
        # if no, create it (download profile)
        # if yes build link


    # check les files
    
    # read the channels.
    with open(ytpro_file, encoding="utf8") as f:
        yt_profile = eval(f.read())
    
    try: 
        # for each channel:
        for channel in yt_profile['brandingSettings']['channel']['featuredChannelsUrls']:
            #cherche si ya dejà un node avec channelID
            yt_featchan_n = graph.nodes.match(channelID=channel).first()
        
            feat_chan_file = write_ytpro_file_fromChannelID(channel)
            with open(feat_chan_file, encoding="utf8") as file:
                feat_chan_pro = eval(file.read())
            
            if not yt_featchan_n:
                # créé node from pro_fil
                # créer node yt_featchan_n    
                yt_user = feat_chan_file.replace(path + "/youtube/","").replace('.ytprofile',"")
                yt_featchan_n = Node("Youtube", user = yt_user)
    
            
            # fill node properties from pro_file:
            for key in keys:
                try:
                    if len(key.split('/'))==1:
                        yt_featchan_n[keys[key]]= feat_chan_pro[key]
                    if len(key.split('/'))==2:
                        yt_featchan_n[keys[key]] = feat_chan_pro[key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
                    if len(key.split('/'))==3:
                        yt_featchan_n[keys[key]] = feat_chan_pro[key.split('/')[len(key.split('/'))-3]][key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
                except:
                    pass
    
        
            source_n = yt_n
            target_n = yt_featchan_n
            rel = Relationship(source_n, "LINKS_TO", target_n)
            rel["yt_type"]="featured_Channel"
            tx = graph.begin()
            tx.merge(yt_featchan_n, "Youtube", "user" )
            tx.merge(rel)
            tx.commit()
            all_rel.append(rel)
            #print("source: ", source_n['user']," target: ",target_n['user'], " rel:", rel)

    except:
        pass
    return(all_rel)
    

In [88]:
# KEEP : This is all Youtube nodes that comes from Website !

for yt_n in graph.nodes.match("Youtube"):
    link_to_yt_n = graph.match(r_type="LINKS_TO", nodes=(None, yt_n))
    for rel in link_to_yt_n:
        if rel.nodes[0].has_label("Website"):
            print(rel.nodes[0]['label'], " -> ", yt_n['title'])

Bourgognemagazine.com  ->  Bourgogne Magazine
Gq.com  ->  GQ
Goldfm.fr  ->  GoldFm33
Glamourparis.com  ->  Glamour Paris
Gulli.fr  ->  Gulli
Leblogalupus.com  ->  None
Leblogdenestor.com  ->  Le Blog de Nestor
Wsws.org  ->  WSWS
Herodote.net  ->  HerodoteVideos
Histoiresordinaires.fr  ->  Webdocumentaires Histoires Ordinaires
Lemediapresse.fr  ->  Le Média
Hitwest.com  ->  hitwestTV
Lepelican-Journal.com  ->  Journal Le Pélican
Humanite.fr  ->  Journal l'Humanité
Zelium.info  ->  Journal Zelium
Infirmiers.com  ->  Infirmiers.com
Lesakerfrancophone.fr  ->  France Saker
Infochretienne.com  ->  Info Chrétienne
Infogm.org  ->  Inf'OGM - Veille citoyenne sur les OGM, les semences et les biotechnologies
Leseconoclastes.fr  ->  Les Econoclastes
Lesjours.fr  ->  Les Jours
Lesmoutonsenrages.fr  ->  Union Populaire Républicaine
Letc.fr  ->  Travailleur Catalan
Journaldemillau.fr  ->  Journal de Millau
Lundi.am  ->  Lundi Matin
Luttedeclasse.org  ->  None
Kontrekulture.com  ->  Kontre Kulture
Lvs

# Continue here


In [92]:

# This builds all featured channels links from youtube nodes that comes from website

for yt_n in graph.nodes.match("Youtube"):
    link_to_yt_n = graph.match(r_type="LINKS_TO", nodes=(None, yt_n))
    for rel in link_to_yt_n:
        if rel.nodes[0].has_label("Website"):
            print(yt_n['url'])
            try:
                push_yt_profile_inNeo4j(yt_n['url'])
                print(push_ytnode_featured_channels_inNeo4j(yt_n))
            except:
                print("ERROR WITH NODE", node)


https://www.youtube.com/channel/UC0JvXUJWwosDRsyjsaoIO4Q
[]
https://www.youtube.com/user/gqvideos/
[(_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2790), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2792), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2793), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2794), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2795), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2796), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2797), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2798), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2799), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2800), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2801), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2802), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2803), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2804), (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2805)]
https://www.you

[]
https://youtube.com/channel/UCGqp3To1Y6IhPU4Pnt9FLYA
2020-03-13 10:37:34.289722 Error 20 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a youtube channelID
2020-03-13 10:37:34.289722 Error 23 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a valid youtube channel
2020-03-13 10:37:34.514195 Error 20 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a youtube channelID
2020-03-13 10:37:34.514195 Error 23 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a valid youtube channel
2020-03-13 10:37:34.516572 Error 22 :  - https://youtube.com/channel/UCGqp3To1Y6IhPU4Pnt9FLYA is not a valid youtube url. Please check the corresponding yt node
2020-03-13 10:37:34.749639 Error 20 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a youtube channelID
2020-03-13 10:37:34.749639 Error 23 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a valid youtube channel
2020-03-13 10:37:35.133313 Error 20 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a youtube channelID
2020-03-13 10:37:35.146712 Error 23 : UCGqp3To1Y6IhPU4Pnt9FLYA is not a valid youtube channel
2020-03-13 10:37:35.146712 Error 22 :  - https:

2020-03-13 10:38:06.854299 Error 20 : UCLnt3x5FAmcJ4PuW6lYuK6w is not a youtube channelID
[(_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2925), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2926), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2927), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2928), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2929), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2930), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2931), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2932), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2933), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2934), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2935), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2936), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2937), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2938), (_1524)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2939), (_1524)-[

[(_1932)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2975), (_1932)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2976), (_1932)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2977), (_1932)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2978)]
https://www.youtube.com/user/iamdieudo4
[]
https://www.youtube.com/channel/UCNgiSaV2F3Cx27Po2QGB_QQ
[]
https://www.youtube.com/channel/UCxjM6SLm5DoZKEQXoj3YvOA
[]
https://www.youtube.com/c/FondationPol%C3%A9mia
[]
https://www.youtube.com/user/EditionsCdP
[]
https://www.youtube.com/channel/UCCDPdHuBGfjMxlM3xZANgGQ
[]
https://www.youtube.com/channel/UCtNyuN_HFlNYO0-gqXwhLTg
[]
https://www.youtube.com/channel/https://www.youtube.com/channel/UC6r2wWG5jF4Ls5MtCj3i0Xw?view_as=subscriber
[]
https://www.youtube.com/user/ELLEfr?sub_confirmation=1
[(_2041)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2979), (_2041)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2074), (_2041)-[:LINKS_TO {yt_type: 'featured_Channel'}]->(_2980), (_2041)-[:LINKS_TO {yt_type: 'feat

[]
https://www.youtube.com/channel/UC_UJKl9Hvn91C22nIDJrUHA?view_as=subscriber
[]
https://www.youtube.com/channel/UCQ3Hc1Jief2ZiLSYfEeIiVA
2020-03-13 10:38:58.293506 Error 20 : UCQ3Hc1Jief2ZiLSYfEeIiVA is not a youtube channelID
2020-03-13 10:38:58.295539 Error 23 : UCQ3Hc1Jief2ZiLSYfEeIiVA is not a valid youtube channel
2020-03-13 10:38:58.549989 Error 20 : UCQ3Hc1Jief2ZiLSYfEeIiVA is not a youtube channelID
2020-03-13 10:38:58.553542 Error 23 : UCQ3Hc1Jief2ZiLSYfEeIiVA is not a valid youtube channel
2020-03-13 10:38:58.553747 Error 22 :  - https://www.youtube.com/channel/UCQ3Hc1Jief2ZiLSYfEeIiVA is not a valid youtube url. Please check the corresponding yt node
2020-03-13 10:38:58.853964 Error 20 : UCQ3Hc1Jief2ZiLSYfEeIiVA is not a youtube channelID
2020-03-13 10:38:58.859241 Error 23 : UCQ3Hc1Jief2ZiLSYfEeIiVA is not a valid youtube channel
2020-03-13 10:38:59.113653 Error 20 : UCQ3Hc1Jief2ZiLSYfEeIiVA is not a youtube channelID
2020-03-13 10:38:59.113653 Error 23 : UCQ3Hc1Jief2ZiLS

In [ ]:
# and build relationships directly from nodes

In [168]:
# THIS HAS TO BE TESTED !!!!!!!!!


def push_ytnode_links_inNeo4j(yt_n):
    all_rel=[]
    
    if yt_n['ytpro_file']: # ça marche pas quand ya watch par exemple
        screen_name = yt_n['ytpro_file'].replace(path + "/youtube/","").replace(".ytprofile","")

        yt_links_file = write_yt_links_file(screen_name)
        with open(yt_links_file, encoding="utf8") as f:
            yt_links = eval(f.read())

        for link in yt_links:
            #Build node
            target_n = PushUrlInfoInNeo4j(link['url'])
            
            # if twitter : cree twitter node
            if getTwitterUsername(link['url'])!="":
                target_n = push_twitpro_in_Neo4j(screen_name)
            
            # if youtube : créé yt node
            if getYouTubeNameAndurl(link['url'])!=('', ''):
                # if node doesnt exist: create it
                if graph.nodes.match(url=url).first()==None:
                    tx = graph.begin()
                    target_n = Node("Youtube", user = yt_user, url = yt_url )
                    tx.merge(target_n, "Youtube", "user" )
                    tx.commit()
                             
                # Push profile info to the node 
                push_yt_profile_inNeo4j(link['url'])
            
            # if facebook : cree fb node
            if getFacebookUsername(link['url'])!="":
                tx = graph.begin()
                fb = getFacebookUsername(link['url'])
                target_n = Node("Facebook", user = fb)
                tx.merge(target_n, "Facebook", "user" )
                tx.commit()
            
            #Build link
            rel = Relationship(yt_n, "LINKS_TO",target_n)
            rel["yt_type"]="link"
            all_rel.append(rel)


In [108]:
# starts from yt_n
screen_name = yt_n['ytpro_file'].replace(path + "/youtube/","").replace(".ytprofile","")

write_yt_links_file(screen_name)
yt_links_file = write_yt_links_file(screen_name)
with open(yt_links_file, encoding="utf8") as f:
    yt_links = eval(f.read())

for link in yt_links:
    
    if getTwitterUsername(link['url'])!="":
        print(link['url'],myUrlParse(link['url']),getTwitterUsername(link['url']))
        #twit_n = push_twitpro_in_Neo4j(screen_name)
    else:
        if getFacebookUsername(link['url'])!="":
            print(link['url'],myUrlParse(link['url']),getFacebookUsername(link['url']))
        else:
            if getYouTubeNameAndurl(link['url'])!=('', ''):
                print(link['url'],myUrlParse(link['url']),getYouTubeNameAndurl(link['url']))
            else:
                print(link['url'],myUrlParse(link['url']) )

https://www.facebook.com/histoires.ordinaires/ facebook.com histoires.ordinaires
https://www.histoiresordinaires.fr/ histoiresordinaires.fr
https://www.les11etsaintperan.info/ les11etsaintperan.info


In [173]:
# KEEP : This is all Youtube nodes that comes from Website !

for yt_n in graph.nodes.match("Youtube").limit(50):
    link_to_yt_n = graph.match(r_type="LINKS_TO", nodes=(None, yt_n))
    for rel in link_to_yt_n:
        if rel.nodes[0].has_label("Website"):
            try:
                print(yt_n['ytpro_file'].replace(path + "/youtube/","").replace(".ytprofile",""))
            except:
                print(yt_n)
            #push_ytnode_links_inNeo4j(yt_n)
            #print("")

bourgogne_magazine
gqvideos
goldfm33
glamourparis
chainegulli
(_521:Youtube {url: 'http://www.youtube.com/watch?v=dtUH2YSFlVU', user: 'watch?v=dtuh2ysflvu'})
le_blog_de_nestor
wsws
herodotevideos
redactionho
lemediaofficiel
hitwesttv
journal_le_pélican
journal_l'humanité
journalzelium
infirmiers.com
france_saker
info_chrétienne
inf'ogm_-_veille_citoyenne_sur_les_ogm,_les_semences_et_les_biotechnologies
leseconoclastesfr
les_jours
uprfr
travailleur_catalan
journal_de_millau
lundi_matin
(_749:Youtube {url: 'https://www.youtube.com/watch?v=p5Ac7ap_MAY', user: 'watch?v=p5ac7ap_may'})
kontre_kulture
leventselevemedia
made_in_marseille
epenser1
bassin_arcachon_la_depeche_du_bassin
l'adn
placeaupeuple
metatv1
mrmondialisation
24heures
(_873:Youtube {url: 'https://www.youtube.com/watch?v=mKKU9QMzBKM', user: 'watch?v=mkku9qmzbkm'})
lenouveaujourj
orient_xxi
fonda30millionsdamis
panafricanmusic
37°_mag
la_sardine_du_port
(_949:Youtube {url: 'https://youtube.com/channel/UCGqp3To1Y6IhPU4Pnt9FLYA',

In [171]:
yt_n

(_1054:Youtube {channelID: 'UCDn6zvBZNEWUv44xjgELicA', commentCount: '0', country: 'FR', description: "Reporterre est le quotidien de l'\u00e9cologie en France. En acc\u00e8s libre, sans publicit\u00e9, sans actionnaire Reporterre vit uniquement gr\u00e2ce aux dons ses lecteurs ! \nPour suivre toute l'actualit\u00e9 \u00e9cologique rendez-vous sur Reporterre.net", hiddenSubscriberCount: false, keywords: 'Quotidien \u00e9cologie', publishedAt: '2013-06-07T08:54:18.000Z', subscriberCount: '4890', title: 'Reporterre. Net', uploadPlaylist: 'UUDn6zvBZNEWUv44xjgELicA', url: 'https://www.youtube.com/channel/UCDn6zvBZNEWUv44xjgELicA', user: 'reporterre.net', videoCount: '174', viewCount: '958651', ytpro_file: 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/reporterre._net.ytprofile'})

In [170]:
url="https://www.mediapart.fr/"
PushUrlInfoInNeo4j(url)

(_1882:Website {alex_rank: 16873, hyphe_date: 1573473167274.0, label: 'Mediapart.fr', pages_crawled: 5586, pages_total: 60815, site_name: 'mediapart.fr', title: "Site d'information fran\u00e7ais d'actualit\u00e9s ind\u00e9pendant et participatif en ligne | Mediapart"})

In [ ]:
def PushRSUrlInfoInNeo4j(url):
    '''From url or sitename, read firstpage, extract SN info and push nodes and relationship in neo4j'''
    
    (site_name, title ) = getFirstPageNode(url)
    siteNodeRes = graph.nodes.match("Website", site_name=site_name)
    sitenodenb = len(siteNodeRes)
    
    if sitenodenb != 1:
        error = str(datetime.datetime.now()) + " Error 9 : " + site_name + " has " + str(sitenodenb) + " nodes in DB"
        print(error)
        with open("error_file.txt", "a", encoding='utf-8') as file:
            file.write(error +"\n")
    else:
        siteNode=siteNodeRes.first()
        (fb,twit,yt_user,yt_url,sc,insta,dm)= getFirstPageRSNode(url)
     
        tx = graph.begin()

        if fb != "":
            fb_n = Node("Facebook", user = fb)
            tx.merge(fb_n, "Facebook", "user" )
            site_fb_r = Relationship(siteNode, "LINKS_TO", fb_n)
            tx.merge(site_fb_r)
        
        if twit != "":
            twit_n = Node("Twitter", user = twit)
            tx.merge(twit_n, "Twitter", "user" )
            site_twit_r = Relationship(siteNode, "LINKS_TO", twit_n)
            tx.merge(site_twit_r)
    
        if yt_user != "":
            yt_n = Node("Youtube", user = yt_user, url = yt_url )
            tx.merge(yt_n, "Youtube", "user" )
            site_yt_r = Relationship(siteNode, "LINKS_TO", yt_n)
            tx.merge(site_yt_r)
    
        tx.commit()
    return(siteNode)


In [163]:
graph.nodes.match("Youtube").first()['test']

In [132]:
graph.nodes.match("Youtube").first()

(_463:Youtube {channelID: 'UC0JvXUJWwosDRsyjsaoIO4Q', commentCount: '0', description: '', hiddenSubscriberCount: false, publishedAt: '2017-12-18T10:14:00.000Z', subscriberCount: '23', title: 'Bourgogne Magazine', uploadPlaylist: 'UU0JvXUJWwosDRsyjsaoIO4Q', url: 'https://www.youtube.com/channel/UC0JvXUJWwosDRsyjsaoIO4Q', user: 'bourgognemagazine.com', videoCount: '15', viewCount: '7011', ytpro_file: 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/bourgogne_magazine.ytprofile'})

In [140]:
bourgogne_n 

(_463:Youtube {channelID: 'UC0JvXUJWwosDRsyjsaoIO4Q', commentCount: '0', description: '', hiddenSubscriberCount: false, publishedAt: '2017-12-18T10:14:00.000Z', subscriberCount: '23', title: 'Bourgogne Magazine', uploadPlaylist: 'UU0JvXUJWwosDRsyjsaoIO4Q', url: 'https://www.youtube.com/channel/UC0JvXUJWwosDRsyjsaoIO4Q', user: 'bourgognemagazine.com', videoCount: '15', viewCount: '7011', ytpro_file: 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/bourgogne_magazine.ytprofile'})

In [151]:
yt_n = Node("Youtube", user = yt_user, url = yt_url )
yt_n['test']="test"
yt_n

(:Youtube {test: 'test', url: 'https://www.youtube.com/channel/UC0JvXUJWwosDRsyjsaoIO4Q', user: 'bourgognemagazine.com'})

In [161]:
tx = graph.begin()
tx.push(bourgogne_n )
tx.commit()

# ICI 

UCKbNOpmhmwBuJXXMYOg_txA
UCftwRNsjfRo08xYE31tkiyw
UCIsbLox_y9dCIMLd8tdC6qg
UCsD-Qms-AkXDrsU962OicLw
UC3TIRH4PPSDde66Hg5wGh-w
UCb0tMboxhHE8Jx6-nhJmRPw
UC0k238zFx-Z8xFH0sxCrPJg
UCr7ZFxNlDvb3waeoTN1_3mA
UCbpMy0Fg74eXXkvxJrtEn3w
UCcjhYlL1WRBjKaJsMH_h7Lg
UCY9eEaH19ulZmdusctU1ejA
UCGiSCVGNukLqv8hwpKCsQKQ
UC7duzDwGoGU6S51Qyask0kA
UCkMOtD7MMYs1H55XH6CkWEw
UCB_TGajnJymiSmu6zWkjW2Q
node: gqvideos


source:  gqvideos
target:  wmagazine
rel: (_471)-[:LINKS_TO {yt_type: 'featured_Channel'}]->()


'            \ntx = graph.begin()\ntx.merge(yt_featchan_n, "Youtube", "user" )\n\n            tx.merge(yt_featchan_n, "Youtube", "user" )\n            site_yt_r = Relationship(siteNode, "LINKS_TO", yt_n)\n            tx.merge(site_yt_r)\ntx.commit()            \n'

'gqsports'

error id
error snippet/title
error snippet/description
error snippet/customUrl
error snippet/publishedAt
error snippet/country
error contentDetails/relatedPlaylists/uploads
error statistics/viewCount
error statistics/commentCount
error statistics/subscriberCount
error statistics/hiddenSubscriberCount
error statistics/videoCount
error brandingSettings/channel/keywords


NameError: name 'yt_featchan_n' is not defined

In [109]:
# pareil pour links
with open(yt_links_file, encoding="utf8") as f:
    yt_links = eval(f.read())
    
for link in yt_links:
    print(link['url'],myUrlParse(link['url']) )
    
    if getTwitterUsername(link['url'])!="":
        print(getTwitterUsername(link['url']))
        #twit_n = push_twitpro_in_Neo4j(screen_name)
    if getFacebookUsername(link['url'])!="":
        print(getFacebookUsername(link['url']))
    if getYouTubeNameAndurl(link['url'])!=('', ''):
        print(getYouTubeNameAndurl(link['url']))
        
    # SI c'est réseau social
        


# et creer FOLLOWS relationships pour subscriptions

http://www.twitter.com/GQMagazine twitter.com
GQMagazine
http://www.facebook.com/gq facebook.com
gq
https://www.instagram.com/gq/ instagram.com
http://www.gq.com/ gq.com


In [222]:
import glob
linkslist={""}
path2 = path+"\\youtube\*.ytlinks"
i=0
linknum = 0
for yt_links_file in glob.glob(path2):
    linknum = linknum+1
    #print(linknum, yt_links_file)
    
    with open(yt_links_file) as f:
        yt_links = eval(f.read())
    for link in yt_links:
        linkslist.add(myUrlParse(link['url']))
        
len(linkslist)
for link in linkslist:
    if graph.nodes.match(site_name=link).first()==None:
        print(link)
        
# Check node et construit
# Creer lien LINKS TO vers site

# créé lien LINKS TO pour les feature channels
# Cree lien FOLLOWS pour subscriptions


# Faire pareil sur twitter avec les twitter profiles

# Inserer les liens Alexa
# Checker les sites Alexa

# Monde diplo
# ACPM

# Tester recherche de site + liens 


discord.gg
les11etsaintperan.info
deezer.com
epinalinos.fr
snapchat.com
boucherie-ovalie.com
facebook.com
play.google.com
youtube.com
tumblr.com
bitly.com
unisciel.fr
linkedin.com
vk.com
spotify.com
telegram.org
upr.fr
apple.co
pearltrees.com
netvibes.com
google.com
beautiful-box.com
tropdimpots.com
erfm.fr
remiremontinfo.fr
dailymotion.com
laplaineinfo.fr
telegram.me
umalis.fr
amazon.com
mesopinions.com_fr
jean-luc-melenchon.fr
plus.google.com
jlm2017.fr
flipboard.com
francemusique.fr
globalresearch.ca
lesenquetesducontribuable.fr
tipeee.com
oummatv.tv
viadeo.com
phantasia.be
taranisnews.com
paypal.com
nathalie-arthaud.info
soundcloud.com
paralleles.org
twitter.com
itunes.apple.com
jemabonne.fr
rakuten.tv
instagram.com
lesoffrescanal.fr
canalplus.fr
wikipedia.org
pinterest.com
tivi5mondeplus.com


In [151]:
yt_links_file
with open(yt_links_file) as f:
    yt_link = f.read()
print(yt_link)

[{'title': 'Twitter', 'url': 'https://twitter.com/AGRI_72'}, {'title': 'Facebook', 'url': 'https://www.facebook.com/AGRI72'}, {'title': 'Notre site', 'url': 'http://www.agri72.fr/'}]


In [112]:
# Trouve noeud. Si existe, builds relationship to channel. Otherwise create it.

In [126]:
#linkslist={"1"}
linkslist.add("2")

In [127]:
print(linkslist)

{'1', '2'}


In [81]:
myUrlParse("http://www.gq.com/")

'gq.com'

In [85]:
for node in graph.nodes.match():
    try:
        if "gq" in node['site_name']:
            print(node)
    except:
        pass

(_465:Website {alex_rank: 5918, hyphe_date: 1574071736175.0, label: 'Gq.com', pages_crawled: 1, pages_total: 56, site_name: 'gq.com', title: "Men's Fashion, Style, Grooming, Fitness, Lifestyle, News & Politics | GQ"})


In [73]:
yt_profile['brandingSettings']['channel']

{'title': 'GQ',
 'description': 'For more than 50 years, GQ has been the premier men’s magazine, providing definitive coverage of style, culture, politics and more. In that tradition, GQ’s video channel covers every part of a man’s life, from entertainment and sports to fashion and grooming advice. So join celebrities from 2 Chainz, Stephen Curry and Channing Tatum to Amy Schumer, Kendall Jenner and Kate Upton for a look at the best in pop culture. Welcome to the modern man’s guide to style advice, dating tips, celebrity videos, music, sports and more.',
 'keywords': 'GQ "men\'s style" "men\'s fashion"',
 'defaultTab': 'Featured',
 'trackingAnalyticsAccountId': 'UA-37483026-26',
 'moderateComments': True,
 'showRelatedChannels': True,
 'showBrowseView': True,
 'featuredChannelsTitle': 'Featured Channels',
 'featuredChannelsUrls': ['UCKbNOpmhmwBuJXXMYOg_txA',
  'UCftwRNsjfRo08xYE31tkiyw',
  'UCIsbLox_y9dCIMLd8tdC6qg',
  'UCsD-Qms-AkXDrsU962OicLw',
  'UC3TIRH4PPSDde66Hg5wGh-w',
  'UCb0tM

In [682]:
# il manque creer les noeuds et connections pour ces channels:
for channel in yt_profile['brandingSettings']['channel']['featuredChannelsUrls']:
    print(channel)



KeyError: 'featuredChannelsUrls'

In [691]:
# creer aussi les liens FOLLOWS sur ces comptes :
with open(ytsub_file, encoding="utf8") as f:
        yt_sub = eval(f.read()) 

for sub in yt_sub:
    print(sub['snippet']['title'])

FRANÇOIS RUFFIN
Brut


In [696]:
# et les liens pour ces liens:
with open(yt_links_file, encoding="utf8") as f:
    yt_links = eval(f.read()) 
for link in yt_links:
    print(link['url'])
print(link['url'])

https://www.fakirpresse.info/boutique/s-abonner/57-abonnement-par-prelevement-automatique.html
https://www.fakirpresse.info/boutique/s-abonner/57-abonnement-par-prelevement-automatique.html


In [44]:
# Virer les trucs qui correspondent à mediapart sur le noeud fakir
yt_n

(_2187:Youtube {url: 'https://www.youtube.com/user/Fakirpresse', user: 'fakirpresse'})

In [699]:
# liste des sites que le youtube node link : avant
for r in graph.match((yt_n,)):
    print(r, r.end_node)

In [700]:
# ça ça chope le noeud qui correspond au site et créé le lien yt vers site
PushRSUrlInfoInNeo4j(link['url'])
tx = graph.begin()
site_yt_r = Relationship(yt_n, "LINKS_TO", PushRSUrlInfoInNeo4j(link['url']))
tx.merge(site_yt_r)
    
tx.commit()

In [701]:
# Et si refait ça :liste des sites que le youtube node link : avant
for r in graph.match((yt_n,)):
    print(r, r.end_node)
    
# C'est bon !!

(_2187)-[:LINKS_TO {}]->(_1156) (_1156:Website {alex_rank: 1342178, hyphe_date: 1574071759441.0, label: 'Fakirpresse.info', pages_crawled: 0, pages_total: 51, site_name: 'fakirpresse.info', title: 'Journal Fakir - Journal f\u00e2ch\u00e9 avec tout le monde. Ou presque.'})


# Refait avec glamourParis


In [51]:
url="https://www.youtube.com/channel/UCSnniCDEE-rapQ6zriW0LfQ"

In [50]:
# trouve noeuds youtube avec cette url
yt_n = graph.nodes.match(url=url).first()
yt_n

(_492:Youtube {channelID: 'UCSnniCDEE-rapQ6zriW0LfQ', commentCount: '0', customUrl: 'glamourparis', description: 'Pop, dr\u00f4le, inform\u00e9e, chaque jour votre dose de Glamour #GlamourNoFilter\n\nwww.glamour.fr\n\n\nVisitez les autres chaines de Cond\u00e9 Nast :\nhttp://www.youtube.com/GQFRANCE\nhttp://www.youtube.com/VOGUEPARISFR\n\nAbonnez-vous au magazine Glamour sur le site:\nhttp://www.glamourparis.com/abonnements', hiddenSubscriberCount: false, keywords: 'glamour mode fashion news people c\u00e9l\u00e9brit\u00e9s beaut\u00e9 glamour.com cond\u00e9 nast', publishedAt: '2007-06-14T14:54:45.000Z', subscriberCount: '25100', title: 'Glamour Paris', uploadPlaylist: 'UUSnniCDEE-rapQ6zriW0LfQ', url: 'https://www.youtube.com/channel/UCSnniCDEE-rapQ6zriW0LfQ', user: 'glamourparis.com', videoCount: '340', viewCount: '12913573'})

In [721]:
# check les files
# Doesn't work because of quota
'''(ytpro_file,ytsub_file,yt_links_file) = write_yt_files_from_yt_url(url)'''
# Try this : 
url="https://www.youtube.com/user/glamourparis"
(ytpro_file,ytsub_file,yt_links_file) = write_yt_files_from_yt_url(url)

In [722]:
with open(ytpro_file, encoding="utf8") as f:
        yt_profile = eval(f.read()) 
yt_profile

{'kind': 'youtube#channel',
 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/JHQ3Bo2KIUijiUM73tiPeDd1zrM"',
 'id': 'UCSnniCDEE-rapQ6zriW0LfQ',
 'snippet': {'title': 'Glamour Paris',
  'description': 'Pop, drôle, informée, chaque jour votre dose de Glamour #GlamourNoFilter\n\nwww.glamour.fr\n\n\nVisitez les autres chaines de Condé Nast :\nhttp://www.youtube.com/GQFRANCE\nhttp://www.youtube.com/VOGUEPARISFR\n\nAbonnez-vous au magazine Glamour sur le site:\nhttp://www.glamourparis.com/abonnements',
  'customUrl': 'glamourparis',
  'publishedAt': '2007-06-14T14:54:45.000Z',
  'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/a/AATXAJwNpsqIgKzpa4zAIy_6rpX8HFeu5TnTImNKqw=s88-c-k-c0xffffffff-no-rj-mo',
    'width': 88,
    'height': 88},
   'medium': {'url': 'https://yt3.ggpht.com/a/AATXAJwNpsqIgKzpa4zAIy_6rpX8HFeu5TnTImNKqw=s240-c-k-c0xffffffff-no-rj-mo',
    'width': 240,
    'height': 240},
   'high': {'url': 'https://yt3.ggpht.com/a/AATXAJwNpsqIgKzpa4zAIy_6rpX8HFeu5TnTImNKqw=s800-c-k-c0xffff

In [723]:
# ca definie les keys et test le bousin en printant
keys = {'id':'channelID', 
        'snippet/title':'title', 
        'snippet/description':'description',
        'snippet/customUrl':'customUrl', 
        'snippet/publishedAt':'publishedAt',
        'snippet/country':'country', 
        'contentDetails/relatedPlaylists/uploads':'uploadPlaylist',
        'statistics/viewCount':'viewCount', 
        'statistics/commentCount':'commentCount',
        'statistics/subscriberCount':'subscriberCount', 
        'statistics/hiddenSubscriberCount':'hiddenSubscriberCount',
        'statistics/videoCount':'videoCount',
        'brandingSettings/channel/keywords':'keywords'
       }
i=0
for key in keys:
    i=i+1
    try:
        if len(key.split('/'))==1:
            print(i,key, yt_profile[key], keys[key])
        if len(key.split('/'))==2:
            print(i,key, yt_profile[key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]],keys[key])
        if len(key.split('/'))==3:
            print(i,key, yt_profile[key.split('/')[len(key.split('/'))-3]][key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]], keys[key])
    except:
        print("error",key )

1 id UCSnniCDEE-rapQ6zriW0LfQ channelID
2 snippet/title Glamour Paris title
3 snippet/description Pop, drôle, informée, chaque jour votre dose de Glamour #GlamourNoFilter

www.glamour.fr


Visitez les autres chaines de Condé Nast :
http://www.youtube.com/GQFRANCE
http://www.youtube.com/VOGUEPARISFR

Abonnez-vous au magazine Glamour sur le site:
http://www.glamourparis.com/abonnements description
4 snippet/customUrl glamourparis customUrl
5 snippet/publishedAt 2007-06-14T14:54:45.000Z publishedAt
error snippet/country
7 contentDetails/relatedPlaylists/uploads UUSnniCDEE-rapQ6zriW0LfQ uploadPlaylist
8 statistics/viewCount 12913573 viewCount
9 statistics/commentCount 0 commentCount
10 statistics/subscriberCount 25100 subscriberCount
11 statistics/hiddenSubscriberCount False hiddenSubscriberCount
12 statistics/videoCount 340 videoCount
13 brandingSettings/channel/keywords glamour mode fashion news people célébrités beauté glamour.com condé nast keywords


In [724]:
# ajoute les trucs dans le node
i=0
for key in keys:
    i=i+1
    try:
        if len(key.split('/'))==1:
            yt_n[keys[key]]= yt_profile[key]
            print(i,key, yt_profile[key], keys[key])
        if len(key.split('/'))==2:
            yt_n[keys[key]] = yt_profile[key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
            print(i,key, yt_profile[key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]],keys[key])
        if len(key.split('/'))==3:
            yt_n[keys[key]] = yt_profile[key.split('/')[len(key.split('/'))-3]][key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]]
            print(i,key, yt_profile[key.split('/')[len(key.split('/'))-3]][key.split('/')[len(key.split('/'))-2]][key.split('/')[len(key.split('/'))-1]], keys[key])
    except:
        print("error",key )

yt_n

1 id UCSnniCDEE-rapQ6zriW0LfQ channelID
2 snippet/title Glamour Paris title
3 snippet/description Pop, drôle, informée, chaque jour votre dose de Glamour #GlamourNoFilter

www.glamour.fr


Visitez les autres chaines de Condé Nast :
http://www.youtube.com/GQFRANCE
http://www.youtube.com/VOGUEPARISFR

Abonnez-vous au magazine Glamour sur le site:
http://www.glamourparis.com/abonnements description
4 snippet/customUrl glamourparis customUrl
5 snippet/publishedAt 2007-06-14T14:54:45.000Z publishedAt
error snippet/country
7 contentDetails/relatedPlaylists/uploads UUSnniCDEE-rapQ6zriW0LfQ uploadPlaylist
8 statistics/viewCount 12913573 viewCount
9 statistics/commentCount 0 commentCount
10 statistics/subscriberCount 25100 subscriberCount
11 statistics/hiddenSubscriberCount False hiddenSubscriberCount
12 statistics/videoCount 340 videoCount
13 brandingSettings/channel/keywords glamour mode fashion news people célébrités beauté glamour.com condé nast keywords


(_492:Youtube {channelID: 'UCSnniCDEE-rapQ6zriW0LfQ', commentCount: '0', customUrl: 'glamourparis', description: 'Pop, dr\u00f4le, inform\u00e9e, chaque jour votre dose de Glamour #GlamourNoFilter\n\nwww.glamour.fr\n\n\nVisitez les autres chaines de Cond\u00e9 Nast :\nhttp://www.youtube.com/GQFRANCE\nhttp://www.youtube.com/VOGUEPARISFR\n\nAbonnez-vous au magazine Glamour sur le site:\nhttp://www.glamourparis.com/abonnements', hiddenSubscriberCount: false, keywords: 'glamour mode fashion news people c\u00e9l\u00e9brit\u00e9s beaut\u00e9 glamour.com cond\u00e9 nast', publishedAt: '2007-06-14T14:54:45.000Z', subscriberCount: '25100', title: 'Glamour Paris', uploadPlaylist: 'UUSnniCDEE-rapQ6zriW0LfQ', url: 'https://www.youtube.com/channel/UCSnniCDEE-rapQ6zriW0LfQ', user: 'glamourparis.com', videoCount: '340', viewCount: '12913573'})

In [725]:
# push le node
tx = graph.begin()
tx.push(yt_n)
tx.commit()
        

In [727]:
url_channel="https://www.youtube.com/channel/UCSnniCDEE-rapQ6zriW0LfQ"
# check le node:
yt_n = graph.nodes.match(url=url_channel).first()
yt_n

(_492:Youtube {channelID: 'UCSnniCDEE-rapQ6zriW0LfQ', commentCount: '0', customUrl: 'glamourparis', description: 'Pop, dr\u00f4le, inform\u00e9e, chaque jour votre dose de Glamour #GlamourNoFilter\n\nwww.glamour.fr\n\n\nVisitez les autres chaines de Cond\u00e9 Nast :\nhttp://www.youtube.com/GQFRANCE\nhttp://www.youtube.com/VOGUEPARISFR\n\nAbonnez-vous au magazine Glamour sur le site:\nhttp://www.glamourparis.com/abonnements', hiddenSubscriberCount: false, keywords: 'glamour mode fashion news people c\u00e9l\u00e9brit\u00e9s beaut\u00e9 glamour.com cond\u00e9 nast', publishedAt: '2007-06-14T14:54:45.000Z', subscriberCount: '25100', title: 'Glamour Paris', uploadPlaylist: 'UUSnniCDEE-rapQ6zriW0LfQ', url: 'https://www.youtube.com/channel/UCSnniCDEE-rapQ6zriW0LfQ', user: 'glamourparis.com', videoCount: '340', viewCount: '12913573'})

In [728]:
# il manque creer les noeuds et connections pour ces channels:
for channel in yt_profile['brandingSettings']['channel']['featuredChannelsUrls']:
    print(channel)

UCli6Q1pd3DitknGEuMKnAVA


In [729]:
# creer aussi les liens FOLLOWS sur ces comptes :
with open(ytsub_file, encoding="utf8") as f:
        yt_sub = eval(f.read()) 

for sub in yt_sub:
    print(sub['snippet']['title'])

Vogue Paris
GQ France
Vanity Fair France


In [732]:
# et les liens pour ces liens:
with open(yt_links_file, encoding="utf8") as f:
    yt_links = eval(f.read()) 
for link in yt_links:
    print(link['url'])
# Pas de liens

In [731]:
yt_links_file

'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/glamourparis.ytlinks'

In [733]:
yt_links

[]

'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/fakirpresse.ytlinks'

{'kind': 'youtube#subscription', 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/7D63mAFQ93OXbUuMA5x6B55cWAY"', 'id': 'r4PZJZlt3Au7MUcilAq7YApy2lq0f6IeI9eHCxxk8vw', 'snippet': {'publishedAt': '2016-12-16T19:07:49.000Z', 'title': 'FRANÇOIS RUFFIN', 'description': 'Chaîne officielle de François Ruffin.', 'resourceId': {'kind': 'youtube#channel', 'channelId': 'UCIQGSp79vVch0vO3Efqif_w'}, 'channelId': 'UCDvIeLuH5wmndQ55cXeKtXw', 'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/-hanI5docBuw/AAAAAAAAAAI/AAAAAAAAAAA/PSvgt50Xx8k/s88-c-k-no-mo-rj-c0xffffff/photo.jpg'}, 'medium': {'url': 'https://yt3.ggpht.com/-hanI5docBuw/AAAAAAAAAAI/AAAAAAAAAAA/PSvgt50Xx8k/s240-c-k-no-mo-rj-c0xffffff/photo.jpg'}, 'high': {'url': 'https://yt3.ggpht.com/-hanI5docBuw/AAAAAAAAAAI/AAAAAAAAAAA/PSvgt50Xx8k/s800-c-k-no-mo-rj-c0xffffff/photo.jpg'}}}, 'contentDetails': {'totalItemCount': 339, 'newItemCount': 0, 'activityType': 'all'}}
{'kind': 'youtube#subscription', 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/fK1Yus8a-CaW5BAzA

In [611]:
for key1 in s.keys():
    print(key1)
    if type(yt_profile[key1])==dict:
        for key2 in yt_profile[key1].keys():
            print("1 ", key2)
            if type(yt_profile[key1][key2])==dict:
                for key3 in yt_profile[key1][key2].keys():
                    print("1 2 ", key3)
                    if type(yt_profile[key1][key2][key3])==dict:
                        for key4 in yt_profile[key1][key2][key3].keys():
                            print("1 2 3 ", key4)
                            if type(yt_profile[key1][key2][key3][key4])==dict:
                                for key5 in yt_profile[key1][key2][key3][key4].keys():
                                    print("1 2 3 4", key5)

kind
etag
id
snippet
1  title
1  description
1  customUrl
1  publishedAt
1  thumbnails
1 2  default
1 2 3  url
1 2 3  width
1 2 3  height
1 2  medium
1 2 3  url
1 2 3  width
1 2 3  height
1 2  high
1 2 3  url
1 2 3  width
1 2 3  height
1  localized
1 2  title
1 2  description
contentDetails
1  relatedPlaylists
1 2  uploads
1 2  watchHistory
1 2  watchLater


kind
etag
id
snippet
   title
   description
   customUrl
   publishedAt
   thumbnails
   localized
   country
contentDetails
   relatedPlaylists
statistics
   viewCount
   commentCount
   subscriberCount
   hiddenSubscriberCount
   videoCount
brandingSettings
   channel
   image
   hints


In [570]:
node = graph.nodes.match(url=url).first()

keys = 

SyntaxError: invalid syntax (<ipython-input-570-5ec25bbb1630>, line 3)

In [ ]:
keys = ['id','name','screen_name','location','description','desc_url','url','followers_count',
                  'friends_count', 'listed_count', 'created_at', 'verified', 'statuses_count']
    
    twitpro_file = write_twitpro_file(screen_name)

for k in keys:
            twitter_n[k] = twit_profile_extracted_keys[k]
        # push in DB
        tx = graph.begin()
        tx.push(twitter_n)
        tx.commit()
        return(twitter_n)

In [512]:
for node in graph.nodes.match("Youtube"):
    url = node['url']
    if ("user" in url) or ("channel" in url) or ("playlist" in url) or ("watch" in url):
        pass
    else:
        if "c/" in url:
            custom_name = url.split("/c/")[1]
        else:
            custom_name = url.split("/")[3]
            
        request = youtube.search().list(
            part="id,snippet",
            maxResults=1,
            q=custom_name,
            type="channel"
            )
        response = request.execute()
        
        try:
            print(custom_name, response['items'][0]['snippet']['title'])
        except:
            print("error with", custom_name)

SEPupload WSWS
journalsatiriqzelium Journal Zelium
24hweb 24heures
results?search_query=riposte+laique JEAN-LUC MÉLENCHON
taranisnews TARANIS NEWS
ajplussaha AJ+ كبريت
aleteiafr AleteiaFR
alimentationgenerale1 avenuedescombattants
unisciel Unisciel
Lopinionfr L'Opinion
lorientlejourTube L'Orient Le Jour
letemps Le Temps
mrlyonmag lyonmag
numerama Numerama
error with FondationPol%C3%A9mia
euractiv EURACTIV
euronews euronews (in English)
mercato Foot Mercato
error with SanteplusmagOfficiel
seloger SeLoger, vous avez les clés
error with TouteleuropeEu
TSAToutsurlAlg%C3%A9rie TSA - Tout sur l'Algérie


'UC0JvXUJWwosDRsyjsaoIO4Q'

In [543]:
i=0
j=0
for node in graph.nodes.match("Youtube"):
    i=i+1
    url=node['url']
    print(url)
    try:
        print(write_yt_files_from_yt_url(url))
    except:
        j=j+1
print(i,j)

https://www.youtube.com/channel/UC0JvXUJWwosDRsyjsaoIO4Q
https://www.youtube.com/user/gqvideos/
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/gqvideos.ytprofile', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/gqvideos.ytsubscriptions', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/gqvideos.ytlinks')
https://www.youtube.com/user/GoldFm33
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/goldfm33.ytprofile', '', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/goldfm33.ytlinks')
https://www.youtube.com/channel/UCSnniCDEE-rapQ6zriW0LfQ
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/glamourparis.ytprofile', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/glamourparis.ytsubscriptions', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/glamourparis.ytlinks')
https://www.youtube.com/channel/UCaIcgxGFjyfZgoIaJT0s0hQ
('C:\\Users\\Jo\\Documents\\Tec

('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/la_sardine_du_port.ytprofile', '', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/la_sardine_du_port.ytlinks')
https://youtube.com/channel/UCGqp3To1Y6IhPU4Pnt9FLYA
https://www.youtube.com/user/thepositivr
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/thepositivr.ytprofile', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/thepositivr.ytsubscriptions', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/thepositivr.ytlinks')
https://youtube.com/channel/UC6eFVS3HetYcMOrak_VALeA
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/la_semaine_du_roussillon.ytprofile', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/la_semaine_du_roussillon.ytsubscriptions', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/la_semaine_du_roussillon.ytlinks')
https://www.youtube.com/user/lozerenouvelle?feature=watch
('C:\\Use

('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/batiramatv.ytprofile', '', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/batiramatv.ytlinks')
https://www.youtube.com/user/lemouv/
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/lemouv.ytprofile', '', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/lemouv.ytlinks')
https://www.youtube.com/user/GlobalResearchTV
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/globalresearchtv.ytprofile', '', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/globalresearchtv.ytlinks')
https://www.youtube.com/user/canalplusfrance
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/canalplusfrance.ytprofile', '', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/canalplusfrance.ytlinks')
https://www.youtube.com/user/MyLittlePlayer
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/mylittleplayer.ytprofil

https://www.youtube.com/channel/UCWKASfAZMU6OrodrDvKJi4A/
https://www.youtube.com/user/Fakirpresse
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/fakirpresse.ytprofile', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/fakirpresse.ytsubscriptions', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/fakirpresse.ytlinks')
https://www.youtube.com/user/FChretienne
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/fchretienne.ytprofile', '', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/fchretienne.ytlinks')
https://www.youtube.com/watch?v=i1aoaDJOcB8
None
https://www.youtube.com/channel/UCanxaCp65Ph4E1sPcEiwRkw/videos
https://www.youtube.com/channel/UCrCQO-y5BgQqH5xVqo47LxQ
https://www.youtube.com/user/MagRollingStone?sub_confirmation=1
('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/magrollingstone.ytprofile', 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/magroll

In [546]:
write_yt_files_from_yt_url(graph.nodes.match("Youtube").first()['url'])

('C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/bourgogne_magazine.ytprofile',
 '',
 'C:\\Users\\Jo\\Documents\\Tech\\ScrapData\\20200114_test/youtube/bourgogne_magazine.ytlinks')

In [547]:
graph.nodes.match("Youtube").first()

(_463:Youtube {url: 'https://www.youtube.com/channel/UC0JvXUJWwosDRsyjsaoIO4Q', user: 'bourgognemagazine.com'})

In [320]:
screen_name = screen_name.lower()
ytpro_file = write_ytpro_file(screen_name)
ytlinks_file = path + "/youtube/" + screen_name + '.ytlinks'
    
if ytpro_file != "":
    with open(ytpro_file, encoding="utf8") as f:
        yt_profile = eval(f.read())
    
about_url = "https://www.youtube.com/channel/" + yt_profile['id'] +"/about"
about_page = urllib.request.urlopen(about_url)
about_soup = BeautifulSoup(about_page)


In [373]:
links = []

In [376]:
for a in about_soup.find_all("a", class_ = "about-channel-link"):
    for ref in a['href'].split('q='):
        if 'http' in ref:
            about_title = a['title']
            about_link = unquote(ref.split('&')[0])
            already=False
            for link in links:
                if link['url']== about_link:
                    already=True
            if not already:
                links.append({'title':about_title,'url':about_link})

In [377]:
links

[{'title': 'Twitter', 'url': 'https://twitter.com/AGRI_72'},
 {'title': 'Facebook', 'url': 'https://www.facebook.com/AGRI72'},
 {'title': 'Notre site', 'url': 'http://www.agri72.fr/'}]

In [370]:
for link in links:
    print(link['url'])

In [225]:
# Si channelID, faire ça d'abord :
channelID = "UC2wrXSZMid7OiCVwCfVpHiA"
ytpro_file = write_ytpro_file_fromChannelID(channelID)

try:
    with open(ytpro_file, encoding="utf8") as f:
        yt_profile = eval(f.read())
    
    try:
        screen_name= yt_profile['snippet']['customUrl']
    except:
        screen_name = response['items'][0]['snippet']['title'].lower().replace(" ", "_")
        
except:
    pass

In [ ]:
screen_name = "lemediaofficiel"

In [226]:
ytsub_file = write_ytsub_file(screen_name)

try:
    with open(ytsub_file, encoding="utf8") as f:
        yt_sub = eval(f.read())
except:
    yt_sub =""

ytvideo_file = write_ytvideos_file(screen_name)

try:
    with open(ytvideo_file, encoding="utf8") as f:
        yt_video = eval(f.read())
except:
    yt_video =""

print(screen_name, " has ", len(yt_sub), " public subscriptions and ", len(yt_video), " videos")



lundi_matin  has  0  public subscriptions and  185  videos


In [246]:
screen_name = "ufcquechoisir"
ytsub_file = write_ytsub_file(screen_name)

try:
    with open(ytsub_file, encoding="utf8") as f:
        yt_sub = eval(f.read())
except:
    yt_sub =""
print(len(yt_sub))


0


In [409]:
screen_name = 'toulouseinfos'
ytsub_file = write_ytsub_file(screen_name)

In [237]:
print("there is ", len(graph.nodes.match("Youtube")), "youtube channels in base")
cha = 0
use = 0

for node in graph.nodes.match("Youtube"):
    if "channel" in str(node['url']):
        cha = cha +1
        print("cha : ",cha)
    else:
        if "user" in str(node['url']):
            use = use +1
            print("use : ", use)
        
        else:
            print(node)
        

    
print(cha, " channels and ", use, "users")

there is  209 youtube channels in base
cha :  1
use :  1
use :  2
cha :  2
cha :  3
(_521:Youtube {url: 'http://www.youtube.com/watch?v=dtUH2YSFlVU', user: 'watch?v=dtuh2ysflvu'})
cha :  4
(_545:Youtube {url: 'http://www.youtube.com/SEPupload', user: 'sepupload'})
cha :  5
use :  3
cha :  6
use :  4
cha :  7
cha :  8
(_618:Youtube {url: 'https://www.youtube.com/journalsatiriqzelium', user: 'journalsatiriqzelium'})
cha :  9
cha :  10
cha :  11
cha :  12
cha :  13
cha :  14
cha :  15
cha :  16
cha :  17
cha :  18
(_749:Youtube {url: 'https://www.youtube.com/watch?v=p5Ac7ap_MAY', user: 'watch?v=p5ac7ap_may'})
cha :  19
cha :  20
cha :  21
use :  5
cha :  22
cha :  23
use :  6
cha :  24
use :  7
(_858:Youtube {url: 'http://www.youtube.com/24hweb', user: '24hweb'})
(_873:Youtube {url: 'https://www.youtube.com/watch?v=mKKU9QMzBKM', user: 'watch?v=mkku9qmzbkm'})
use :  8
cha :  25
use :  9
cha :  26
cha :  27
cha :  28
cha :  29
use :  10
cha :  30
use :  11
use :  12
cha :  31
cha :  32
cha 

In [439]:
# TODO WITH QUOTA :
# 1
for node in graph.nodes.match("Youtube"):
    if "user" in str(node['url']):
        screen_name = node['user'].replace("?feature=watch", "").replace("/videos","").replace('?sub_confirmation=1','').replace("?feature=guide","")
        ytsub_file = write_ytsub_file(screen_name)
        #ytvideo_file = write_ytvideos_file(screen_name)
        try:
            with open(ytsub_file, encoding="utf8") as f:
                yt_sub = eval(f.read())
        except:
            yt_sub =""
            '''
        try:
            with open(ytvideo_file, encoding="utf8") as f:
                yt_video = eval(f.read())
        except:
            yt_video =""
            '''
        write_yt_links_file(screen_name)
        with open(write_yt_links_file(screen_name), encoding="utf8") as f:
            links = eval(f.read())
        print(screen_name, " has ", len(yt_sub), " public subscriptions and ", len(links)," links" )

gqvideos  has  25  public subscriptions and  4  links
goldfm33  has  0  public subscriptions and  1  links
redactionho  has  0  public subscriptions and  3  links
hitwesttv  has  30  public subscriptions and  2  links
epenser1  has  0  public subscriptions and  6  links
placeaupeuple  has  36  public subscriptions and  5  links
mrmondialisation  has  62  public subscriptions and  4  links
lenouveaujourj  has  0  public subscriptions and  0  links
fonda30millionsdamis  has  7  public subscriptions and  4  links
thepositivr  has  158  public subscriptions and  5  links
lozerenouvelle  has  0  public subscriptions and  1  links
prhenrijoyeux  has  0  public subscriptions and  4  links
journalledevoir  has  0  public subscriptions and  4  links
agri72tv  has  0  public subscriptions and  3  links
rtenfrancais  has  0  public subscriptions and  4  links
socialtertv  has  30  public subscriptions and  1  links
topitotv  has  497  public subscriptions and  6  links
jtoulousaintv  has  0  publ

In [440]:
# 2 
for node in graph.nodes.match("Youtube"):
    if "channel" in str(node['url']):
        channelID = node['url'].replace('https://www.youtube.com/channel/',"").replace('https://youtube.com/channel/',"").replace("/featured","")[0:24]

        ytpro_file = write_ytpro_file_fromChannelID(channelID)
        try:
            with open(ytpro_file, encoding="utf8") as f:
                yt_profile = eval(f.read())
    
            try:
                screen_name= yt_profile['snippet']['customUrl']

            except:
                screen_name = yt_profile['snippet']['title'].lower().replace(" ", "_")

        except:
            print("error with channelID ", channelID)
        
        ytsub_file = write_ytsub_file(screen_name)

        try:
            with open(ytsub_file, encoding="utf8") as f:
                yt_sub = eval(f.read())
        except:
            yt_sub =""

        
        write_yt_links_file(screen_name)
        with open(write_yt_links_file(screen_name), encoding="utf8") as f:
            links = eval(f.read())
            
        print(screen_name, " has ", len(yt_sub), " public subscriptions and ", len(links)," links" )
        

bourgogne_magazine  has  0  public subscriptions and  0  links
glamourparis  has  3  public subscriptions and  0  links
chainegulli  has  32  public subscriptions and  6  links
le_blog_de_nestor  has  0  public subscriptions and  0  links
herodotevideos  has  0  public subscriptions and  1  links
lemediaofficiel  has  9  public subscriptions and  5  links
journal_le_pélican  has  0  public subscriptions and  2  links
journal_l'humanité  has  18  public subscriptions and  5  links
infirmiers.com  has  2  public subscriptions and  0  links
france_saker  has  4  public subscriptions and  0  links
info_chrétienne  has  0  public subscriptions and  5  links
inf'ogm_-_veille_citoyenne_sur_les_ogm,_les_semences_et_les_biotechnologies  has  0  public subscriptions and  0  links
leseconoclastesfr  has  1  public subscriptions and  0  links
les_jours  has  0  public subscriptions and  2  links
uprfr  has  0  public subscriptions and  4  links
travailleur_catalan  has  5  public subscriptions and

In [443]:
# 3
for node in graph.nodes.match("Youtube"):
    if not ("channel") in str(node['url']):
        if not ("user") in str(node['url']):
            if not "watch" in str(node['url']):
                if not "playlist" in str(node['url']):
                    if not "%" in str(node['url']):
                        if not "result" in str(node['url']):
                            screen_name = node['user']
                            ytsub_file = write_ytsub_file(screen_name)
                            try:
                                with open(ytsub_file, encoding="utf8") as f:
                                    yt_sub = eval(f.read())
                            except:
                                yt_sub =""
                                
                            write_yt_links_file(screen_name)
                            try:
                                links=""
                                with open(write_yt_links_file(screen_name), encoding="utf8") as f:
                                    links = eval(f.read())
                            except:
                                pass
                            print(screen_name, " has ", len(yt_sub), " public subscriptions and ", len(links)," links" )

sepupload  has  0  public subscriptions and  1  links
journalsatiriqzelium  has  0  public subscriptions and  5  links
24hweb  has  0  public subscriptions and  3  links
taranisnews  has  0  public subscriptions and  3  links
2020-03-04 17:28:26.190750 Error 19 : ajplussaha is not a youtube user
2020-03-04 17:28:26.346919 Error 19 : ajplussaha is not a youtube user
2020-03-04 17:28:26.476420 Error 19 : ajplussaha is not a youtube user
ajplussaha  has  0  public subscriptions and  0  links
2020-03-04 17:28:26.606538 Error 19 : aleteiafr is not a youtube user
2020-03-04 17:28:26.750533 Error 19 : aleteiafr is not a youtube user
2020-03-04 17:28:26.889474 Error 19 : aleteiafr is not a youtube user
aleteiafr  has  0  public subscriptions and  0  links
2020-03-04 17:28:27.023111 Error 19 : alimentationgenerale1 is not a youtube user
2020-03-04 17:28:27.145965 Error 19 : alimentationgenerale1 is not a youtube user
2020-03-04 17:28:27.273747 Error 19 : alimentationgenerale1 is not a youtube u

In [428]:
for node in graph.nodes.match("Youtube"):
    if not ("channel") in str(node['url']):
        if not ("user") in str(node['url']):
            if not "watch" in str(node['url']):
                if not "playlist" in str(node['url']):
                    if not "%" in str(node['url']):
                        if not "result" in str(node['url']):
                            print(node)

(_545:Youtube {url: 'http://www.youtube.com/SEPupload', user: 'sepupload'})
(_618:Youtube {url: 'https://www.youtube.com/journalsatiriqzelium', user: 'journalsatiriqzelium'})
(_858:Youtube {url: 'http://www.youtube.com/24hweb', user: '24hweb'})
(_1217:Youtube {url: 'http://www.youtube.com/taranisnews', user: 'taranisnews'})
(_1272:Youtube {url: 'https://www.youtube.com/ajplussaha', user: 'ajplussaha'})
(_1295:Youtube {url: 'https://www.youtube.com/aleteiafr', user: 'aleteiafr'})
(_1305:Youtube {url: 'https://www.youtube.com/c/alimentationgenerale1', user: 'alimentationgenerale1'})
(_1311:Youtube {url: 'http://www.youtube.com/unisciel', user: 'unisciel'})
(_1456:Youtube {url: 'https://www.youtube.com/Lopinionfr', user: 'lopinionfr'})
(_1469:Youtube {url: 'https://www.youtube.com/lorientlejourTube', user: 'lorientlejourtube'})
(_1477:Youtube {url: 'https://www.youtube.com/letemps', user: 'letemps'})
(_1493:Youtube {url: 'http://www.youtube.com/mrlyonmag', user: 'mrlyonmag'})
(_1806:Youtu

In [430]:
screen_name = "numerama"
request = youtube.channels().list(
    part="id",
    forUsername= screen_name
    )
response = request.execute()
response

{'kind': 'youtube#channelListResponse',
 'etag': '"SJZWTG6xR0eGuCOh2bX6w3s4F94/L8bcJLMDsp1-tMOWe5LN6I8YArU"',
 'pageInfo': {'totalResults': 0, 'resultsPerPage': 5},
 'items': []}

In [445]:
for node in graph.nodes.match("Youtube"):
    print(node['url'])

https://www.youtube.com/channel/UC0JvXUJWwosDRsyjsaoIO4Q
https://www.youtube.com/user/gqvideos/
https://www.youtube.com/user/GoldFm33
https://www.youtube.com/channel/UCSnniCDEE-rapQ6zriW0LfQ
https://www.youtube.com/channel/UCaIcgxGFjyfZgoIaJT0s0hQ
http://www.youtube.com/watch?v=dtUH2YSFlVU
https://www.youtube.com/channel/UCj0HNkgz8eikcMM81NA4CEA
http://www.youtube.com/SEPupload
https://www.youtube.com/channel/UC8ghFWok7LY5oCaSmANBAYQ
http://www.youtube.com/user/redactionHo
https://www.youtube.com/channel/UCT67YOMntJxfRnO_9bXDpvw
https://www.youtube.com/user/hitwestTV
https://www.youtube.com/channel/UCtH9m8VmyPLvIVtv-CcA70A
https://www.youtube.com/channel/UCar0yo-PpeMk51m116826pQ
https://www.youtube.com/journalsatiriqzelium
https://www.youtube.com/channel/UCwmXM0NVeQJ9GOLxiWGgS4g
https://www.youtube.com/channel/UCAR5RuPpdE9FAJTwnhfEnUw
https://www.youtube.com/channel/UCcc5RQ3SAzQ1zCzcEK2W7UQ
https://www.youtube.com/channel/UClPDqnR_sMicLWjpRLLPgTw
https://www.youtube.com/channel/UCDfdKp

In [270]:
# Que faire pour les watch ?

for node in graph.nodes.match("Youtube"):
    if "channel" in str(node['url']):
        pass
    else:
        if "user" in str(node['url']):
            pass
        else:
            if "watch" in str(node['url']):
                print(node['url'],node['user'])

http://www.youtube.com/watch?v=dtUH2YSFlVU watch?v=dtuh2ysflvu
https://www.youtube.com/watch?v=p5Ac7ap_MAY watch?v=p5ac7ap_may
https://www.youtube.com/watch?v=mKKU9QMzBKM watch?v=mkku9qmzbkm
https://www.youtube.com/watch?v=XwJGWea4fyM watch?v=xwjgwea4fym
https://www.youtube.com/watch?v=zMtmZQ4lb2g watch?v=zmtmzq4lb2g
https://www.youtube.com/watch?v=h4qrUGSMjLE watch?v=h4qrugsmjle
https://www.youtube.com/watch?v=VASywEuqFd8 watch?v=vasyweuqfd8
https://www.youtube.com/watch?v=0RRje6d358A&t=247s watch?v=0rrje6d358a&t=247s
https://www.youtube.com/watch?v=k9DBIYFH4v0&feature=emb_title watch?v=k9dbiyfh4v0&feature=emb_title
https://www.youtube.com/watch?v=i1aoaDJOcB8 watch?v=i1aoadjocb8
https://www.youtube.com/watch?v=8shY9slAjVE watch?v=8shy9slajve
https://www.youtube.com/watch?v=Hb4g6YZ39YE watch?v=hb4g6yz39ye


In [288]:
for node in graph.nodes.match("Twitter"):
    if "youtube" in str(node).lower():
        print(node)

(_481:Twitter {created_at: 'Wed Feb 11 01:18:18 +0000 2009', description: 'What you need to know. Follow us on Facebook, Instagram, and YouTube. Visit our home page for the top stories of the day.', followers_count: 2830533, friends_count: 272, id: 20562637, listed_count: 32507, location: 'New York, NY', name: 'Business Insider', screen_name: 'businessinsider', statuses_count: 617551, tw_id: '20562637', url: 'http://businessinsider.com/', user: 'businessinsider', verified: true})
(_760:Twitter {created_at: 'Tue Jan 15 14:23:46 +0000 2008', description: "Un regard humain sur l'actualit\u00e9. Le journal La Croix est pr\u00e9sent sur Facebook | Instagram | YouTube | LinkedIn", followers_count: 59027, friends_count: 1230, id: 12272292, listed_count: 1582, location: '', name: 'La Croix', screen_name: 'LaCroix', statuses_count: 142828, tw_id: '12272292', url: 'http://www.la-croix.com', user: 'lacroix', verified: true})
(_1490:Twitter {created_at: 'Thu Jun 12 09:34:25 +0000 2008', descriptio

# Whois

In [16]:
import subprocess


In [149]:
for node in graph.nodes.match("Website"):
    site_name = node["site_name"]
    getWhois(site_name)


guadeloupe.franceantilles.fr : already file :  709
martinique.franceantilles.fr : already file :  709
gala.fr : already file :  715
gamekult.com : already file :  8993
gentside.com : already file :  8916
geo.fr : already file :  2594
gq.com : already file :  11026
goldfm.fr : already file :  2870
glamourparis.com : already file :  10270
grazia.fr : already file :  2385
guide-piscine.fr : already file :  2657
gulli.fr : already file :  2824
gynecologie-pratique.com : already file :  8841
hautegironde.fr : already file :  2857
hauteloirefootball.fr : already file :  2629
hautesaintonge.fr : already file :  2688
hebdo-ardeche.fr : already file :  2346
hebdo-des-savoie.com : already file :  10003
herault-tribune.com : already file :  13291
herodote.net : already file :  9049
histoires-paranormales.fr : already file :  2398
histoiresordinaires.fr : already file :  2792
historia.fr : already file :  2693
hitwest.com : already file :  9303
hoaxbuster.com : already file :  9064
huffingtonpost.

contexte.com : downloaded whois:  3331
cric-grenoble.info : already file :  8705
cuisine-et-sante.net : already file :  13226
darons.net : already file :  13210
debunkersdehoax.org : already file :  12000
contre-info.com : already file :  13209
demotivateur.fr : already file :  2796
desarmons.net : already file :  11073
diabetologie-pratique.com : already file :  8749
dijonbeaunemag.fr : already file :  2635
dijoncapitale.fr : already file :  2637
dijoncter.info : already file :  9792
cqfd-journal.org : downloaded whois:  2582
diatala.com : downloaded whois:  3303
diktacratie.com : already file :  9982
disclose.ngo : already file :  158
direct-editeurs.fr : already file :  3017
dna.fr : already file :  2223
dossierfamilial.com : already file :  158
dreuz.info : already file :  2352
dieudosphere.com : already file :  158
doctissimo.fr : already file :  3011
dordognelibre.fr : already file :  2843
dw.com_fr : already file :  161
dystopeek.fr : already file :  720
e-marchespublics.com : a

lesmoutonsrebelles.com : already file :  13243
nordpresse.be : already file :  2582
noutous.fr : already file :  2424
nouveaujourj.fr : already file :  2773
oip.org : already file :  2478
oise-agricole.fr : already file :  2902
oisehebdo.fr : already file :  2476
onzemondial.com : already file :  13231
opex360.com : already file :  11039
actu.orange.fr : already file :  695
orientxxi.info : already file :  8473
owni.fr : already file :  2800
pamac.fr : already file :  2857
pan-african-music.com : already file :  9103
paris-luttes.info : already file :  9795
parstoday.com_fr : already file :  161
paysantarnais.com : already file :  13225
penserlibrement.fr : already file :  2749
lepetitquotidien.playbacpresse.fr : already file :  714
pluxactu.com : already file :  9941
poivron.org : already file :  9259
positivr.fr : already file :  2956
premiere.fr : already file :  2758
present.fr : already file :  2549
presse-agricole.net : already file :  13230
pressegalactique.com : already file : 

parents.fr : already file :  2709
patriote.info : already file :  9319
pausecafein.fr : already file :  2782
paysandumidi.fr : already file :  2537
paysansdelaloire.fr : already file :  2669
parisvox.info : already file :  2403
paysdechateaubriant.fr : already file :  2384
pediatrie-pratique.com : already file :  13361
petitbleu.fr : already file :  2959
peuple-libre.fr : already file :  2542
philomag.com : already file :  9190
placegrenet.fr : already file :  3157
planet.fr : already file :  2344
pleinevie.fr : already file :  2823
peupledefrance.com : already file :  8346
pointdevue.fr : already file :  2405
polemia.com : already file :  10278
politico.com : already file :  16286
pourquoidocteur.fr : already file :  2607
presse-bisontine.fr : already file :  2413
pharedere.com : already file :  13365
politis.fr : already file :  2402
presse-citron.net : already file :  13292
presseocean.fr : already file :  2765
programme-television.org : already file :  8094
programme-tv.net : alrea

bravepatrie.com : downloaded whois:  3326
breizatao.com : already file :  12174
breizh-info.com : downloaded whois:  3300
bretons.bzh : already file :  158
brief.me : downloaded whois:  2497
bugeycotiere.fr : already file :  3015
bourgognemagazine.com : already file :  10083
bulletindespalion.fr : already file :  2623
brochuresenligne.com : already file :  13250
businessimmo.com : already file :  9295
businessinsider.com : already file :  12410
buzzfil.me : already file :  9520
buzzly.fr : already file :  2576
bvoltaire.fr : already file :  2745
cadredeville.com : downloaded whois:  3326
cahiersdufootball.net : downloaded whois:  3318
cameditsport.com : already file :  7721
canardpc.com : downloaded whois:  3326
cardiologie-pratique.com : already file :  13357
causette.fr : already file :  2779
causeur.fr : already file :  2584
cbnews.fr : already file :  2794
ccm.net : already file :  8964
centre-presse.fr : already file :  2833
mindnews.fr : already file :  2758
frustrationmagazine.f

In [17]:
def getWhois(site_name):
    ''' From url or sitename, download or read whois info'''
    
    # exception for noScrapSites list
    if site_name in NoScrapSites:
        print(site_name,": noscrap")
        return
    try:
        # if whois fiule already downloaded
        with open(site_name + ".whois", "r", encoding='utf-8') as page:
            print(site_name,": already file : ",len(page.read()))
            return(page)
    
    except:
        result = subprocess.run([r"C:\Program Files\WhoIs\whois64.exe",site_name],capture_output=True, text = True)
        with open(site_name + ".whois", "w", encoding='utf-8') as file:
                file.write(result.stdout)
        print(site_name,": downloaded whois: ",len(result.stdout))
        time.sleep(1) 
        return(result.stdout)
    

In [ ]:
# Parse Whois

In [27]:
randrange(0,len(graph.nodes.match("Website")))

260

In [28]:
def randomWebNode():
    seed = randrange(0,len(graph.nodes.match("Website")))
    return(list(graph.nodes.match("Website"))[seed])

In [60]:
node = randomWebNode()

In [136]:
node = randomWebNode()
site_name = node["site_name"]
with open(site_name + ".whois", "r", encoding='utf-8') as page:
    whois = page.read()
print(site_name,",",len(whois),"characters,",len(whois.split("\n"))," lines,", len(whois.split(site_name))-1,"site-names")

cadredeville.com , 3326 characters, 118  lines, 0 site-names


In [137]:
nicparselist2 = ["type:", "contact:", "address:", "country:", "phone:", "e-mail:", "registrar:", "anonymous:"]
nicparselist3 = ["Name:", "Organization:","Street:","City:", "State/Province:", "Postal Code:", "Country:", "Phone:", "Email:"]
try:
    print("domain:", whois.lower().split("domain:")[1].split("\n")[0])
    print("created", whois.lower().split("created:")[1].split("\n")[0])
except:
    pass
try:
    print("Creation Date:", whois.lower().split("creation date:")[1].split("\n")[0])
    print("Registrant Country:", whois.split("Registrant Country:")[1].split("\n")[0])
except:
    pass
    
for i in range(0, len(whois.split("nic-hdl:"))-1):
    print(i, "nic-hdl:", whois.split("nic-hdl:")[i+1].split("\n")[0])
    whoispart = whois.split("nic-hdl:")[i+1]
    for word in nicparselist2:
        for j in range(0, len(whoispart.split(word))-1):
            print(i, word, whoispart.split(word)[j+1].split("\n")[0])

try:
    print("Domain Name:", whois.split("Domain Name:")[1].split("\n")[0])
    whois_good = whois.split("Domain Name:")[1]            
    for k in range(0, len(whois_good.split("ID:"))-1):
        whoispart = whois_good.split("ID:")[k+1]
        for word in nicparselist3:
            for j in range(0, len(whoispart.split(word))-1):
                print(k, word, whoispart.split(word)[j+1].split("\n")[0])
except:
    pass

Creation Date:  2003-12-04t14:49:39z


In [138]:
print(whois)
        


Whois v1.21 - Domain information lookup
Copyright (C) 2005-2019 Mark Russinovich
Sysinternals - www.sysinternals.com

Connecting to COM.whois-servers.net...

WHOIS Server: whois.gandi.net

   Registrar URL: http://www.gandi.net

   Updated Date: 2019-12-04T10:44:50Z

   Creation Date: 2003-12-04T14:49:39Z

   Registry Expiry Date: 2021-12-04T14:49:39Z

   Registrar: Gandi SAS

   Registrar IANA ID: 81

   Registrar Abuse Contact Email: abuse@support.gandi.net

   Registrar Abuse Contact Phone: +33.170377661

   Domain Status: clientTransferProhibited https://icann.org/epp#clientTransferProhibited

   Name Server: NS-174-C.GANDI.NET

   Name Server: NS-238-B.GANDI.NET

   Name Server: NS-68-A.GANDI.NET

   DNSSEC: unsigned

   URL of the ICANN Whois Inaccuracy Complaint Form: https://www.icann.org/wicf/

>>> Last update of whois database: 2020-04-02T22:20:25Z <<<



For more information on Whois status codes, please visit https://icann.org/epp



NOTICE: The expiration date displayed i

In [147]:
i=0
for node in graph.nodes.match("Website"):
    site_name = node["site_name"]
    with open(site_name + ".whois", "r", encoding='utf-8') as page:
        whois = page.read()
    if len(whois.split("Your IP has been restricted"))>1:
        i=i+1
        print(i, site_name)

        

1 infochretienne.com
2 infogm.org
3 lapressedevesoul.com
4 laquadrature.net
5 lepelerin.com
6 lepostillon.org
7 contexte.com
8 cqfd-journal.org
9 diatala.com
10 samizdat.net
11 tv5monde.com
12 tariqramadan.com
13 transrural-initiatives.org
14 usbeketrica.com
15 campus.la-croix.com
16 spiil.org
17 lindependant.net
18 maxisciences.com
19 nouvelobs.com
20 ohmymag.com
21 ozap.com
22 panamza.com
23 parismatch.com
24 passerelleco.info
25 quechoisir.org
26 rba.fm
27 ruedesjoueurs.com
28 sagefemme-pratique.com
29 santenatureinnovation.com
30 studyrama.com
31 18dumois.info
32 1jour1actu.com
33 acrimed.org
34 action-agricole-picarde.com
35 attac.org
36 arte.tv
37 avenir52.com
38 arteradio.com
39 auvergne-agricole.com
40 be.com
41 bienpublic.com
42 bravepatrie.com
43 breizh-info.com
44 brief.me
45 cadredeville.com
46 cahiersdufootball.net
47 canardpc.com


In [150]:
i=0
for node in graph.nodes.match("Website"):
    site_name = node["site_name"]
    with open(site_name + ".whois", "r", encoding='utf-8') as page:
        whois = page.read()
    if len(whois.split("Your IP has been restricted"))>1:
        i=i+1
        print(i, site_name)
        #os.remove(site_name + ".whois")

1 infochretienne.com
2 infogm.org
3 lapressedevesoul.com
4 laquadrature.net
5 lepelerin.com
6 lepostillon.org
7 contexte.com
8 cqfd-journal.org
9 diatala.com
10 samizdat.net
11 tv5monde.com
12 tariqramadan.com
13 campus.la-croix.com
14 nouvelobs.com
15 ohmymag.com
16 ozap.com
17 panamza.com
18 parismatch.com
19 passerelleco.info
20 quechoisir.org
21 rba.fm
22 ruedesjoueurs.com
23 sagefemme-pratique.com
24 santenatureinnovation.com
25 studyrama.com
26 18dumois.info
27 1jour1actu.com
28 acrimed.org
29 action-agricole-picarde.com
30 attac.org
31 arte.tv
32 avenir52.com
33 arteradio.com
34 auvergne-agricole.com
35 be.com
36 bienpublic.com
37 bravepatrie.com
38 breizh-info.com
39 brief.me
40 cadredeville.com
41 cahiersdufootball.net
42 canardpc.com


In [1198]:
# BAD IDEA !!!!!

In [1233]:
def write_twitfriendsfile(screen_name, cursor=-1, over5000=False):
    '''Get IDs of the accounts a given account follows
    over5000 allows to collect more than 5000 friends'''
    
    screen_name = screen_name.lower()
    # If no file, download :
    cursor = -1
    twitfriends_file = path + "/twitter/" + screen_name + '.twitfriends'
    
    if not Path(twitfriends_file).is_file():
        try:
            ids = []
            r = respectful_api_request(
                'friends/ids', {'screen_name': screen_name, 'cursor': cursor})

        # todo: wait if api requests are exhausted

            if 'errors' in r.json():
                if r.json()['errors'][0]['code'] == 34:
                    return(ids)

            for item in r:
                if isinstance(item, int):
                    ids.append(item)
                elif 'message' in item:
                    print('{0} ({1})'.format(item['message'], item['code']))

            if over5000:
                if 'next_cursor' in r.json():
                    if r.json()['next_cursor'] != 0:
                        ids = ids + collect_friends(screen_name, r.json()['next_cursor'])
                
                
            with open(twitfriends_file, "w", encoding='utf-8') as file:
                    file.write(str(ids)) # Check here if good !!!
            return(twitfriends_file)
            
        except:
            error = str(datetime.datetime.now()) + " Error 18 : " + screen_name + " is not a twitter user"
            print(error)
            with open("error_file.txt", "a", encoding='utf-8') as file:
                file.write(error +"\n")
            return("")
    else:
        return(twitpro_file)   
    
    

In [1231]:
len(collect_friends_fromscreen("mediapart"))

1796

In [1236]:
# BAD IDEA  !!!!!
configurer le timer 

[]

In [ ]:
for node in graph.nodes.match("Twitter"):
    print(node['user'])
    write_twitfriendsfile(node['user'])

gentside
journaldubugey
voici
bourgognemagoff
geofr
gqmagazine
vosgesmatin
businessimmo
goldfm


2020-02-20 15:45:50.631664 Error 18 : goldfm is not a twitter user
businessinsider
glamourparis
Hit the API limit. Waiting for refresh at 2020-02-20T19:50:03Z.
2020-02-20 15:50:03.339226 Error 18 : glamourparis is not a twitter user
lcp
grazia_fr
wedemain
le1hebdo
guidepiscinefr
gulli
leblogalupus
weo_tv
gynecologie_fr
marion_nestor
hauteloirefoot
wsws_updates
ldmocrate
hebdoardeche
enfantsdurhone
Hit the API limit. Waiting for refresh at 2020-02-20T20:05:05Z.


In [182]:
df2=pd.read_csv ('C:/Users/Jo/Nextcloud2/Documents/tech/11Medialist/2811NewListD2_crawls.csv')

In [183]:
df2

,webentity_name,webentity_id,crawljob_id,_id,max_depth,start_urls,user_agent,cookies,phantom,globalStatus,...,created_at,scheduled_at,started_at,crawled_at,finished_at,durationOfCrawl,durationTotal,follow_prefixes,nofollow_prefixes,discover_prefixes
0,Lesinrocks.com,173,78d2c3eb09eb11eaa4520242ac120003,6e70598c-09eb-11ea-b9de-0242ac120004,2,https://www.lesinrocks.com,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,NaN,False,INDEXING,...,2019-11-18T10:08:56.304Z,2019-11-18T10:09:13.861Z,2019-11-18T10:09:20.679Z,2019-11-18T10:20:48.669Z,NaN,-1.574072e+09,-1.574072e+09,s:https|h:com|h:lesinrocks| s:https|h:com|h:le...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
1,Lesechosdutouquet.fr,174,7966d9b009eb11eab9f10242ac120003,6e75723d-09eb-11ea-8b3d-0242ac120004,2,https://www.lesechosdutouquet.fr,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,NaN,False,INDEXING,...,2019-11-18T10:08:56.337Z,2019-11-18T10:09:14.904Z,2019-11-18T10:09:20.679Z,2019-11-18T10:48:37.726Z,NaN,-1.574072e+09,-1.574072e+09,s:https|h:fr|h:lesechosdutouquet| s:https|h:fr...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
2,Lesmutins.org,175,79f0a47d09eb11eab6cc0242ac120003,6e7b939c-09eb-11ea-9e68-0242ac120004,2,https://www.lesmutins.org,Mozilla/5.0 (Windows NT 6.4; WOW64) AppleWebKi...,NaN,False,INDEXING,...,2019-11-18T10:08:56.377Z,2019-11-18T10:09:15.853Z,2019-11-18T10:09:20.679Z,2019-11-18T10:19:18.354Z,NaN,-1.574072e+09,-1.574072e+09,s:https|h:org|h:lesmutins| s:https|h:org|h:les...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
3,Lesoir.be,177,7b2da80009eb11ea88d50242ac120003,6e8b0411-09eb-11ea-bebe-0242ac120004,2,https://www.lesoir.be,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,NaN,False,INDEXING,...,2019-11-18T10:08:56.478Z,2019-11-18T10:09:17.842Z,2019-11-18T10:09:36.516Z,2019-11-18T13:30:09.789Z,NaN,-1.574072e+09,-1.574072e+09,s:https|h:be|h:lesoir| s:https|h:be|h:lesoir|h...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
4,Lessor38.fr,178,7c02266609eb11ea83320242ac120003,6e91bc4c-09eb-11ea-bf6f-0242ac120004,2,https://www.lessor38.fr,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,NaN,False,INDEXING,...,2019-11-18T10:08:56.523Z,2019-11-18T10:09:19.204Z,2019-11-18T10:09:36.516Z,2019-11-18T10:25:58.687Z,NaN,-1.574072e+09,-1.574072e+09,s:https|h:fr|h:lessor38| s:https|h:fr|h:lessor...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
5,Lesnumeriques.com,176,7a91652609eb11ea941c0242ac120003,6e8493c0-09eb-11ea-bd18-0242ac120004,2,https://www.lesnumeriques.com,Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.3...,NaN,False,INDEXING,...,2019-11-18T10:08:56.436Z,2019-11-18T10:09:16.863Z,2019-11-18T10:09:36.516Z,2019-11-18T11:30:08.993Z,NaN,-1.574072e+09,-1.574072e+09,s:https|h:com|h:lesnumeriques| s:https|h:com|h...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
6,guadeloupe.Franceantilles.fr,1,d6ee0a75015411ea9cda0242ac120003,d64b0e0a-0154-11ea-8489-0242ac120004,2,https://www.guadeloupe.franceantilles.fr,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...,NaN,False,ACHIEVED,...,2019-11-07T11:50:47.234Z,2019-11-07T11:50:48.424Z,2019-11-07T11:50:50.408Z,2019-11-07T12:31:03.063Z,2019-11-07T12:31:13.118Z,2.422710e+03,2.424694e+03,s:https|h:fr|h:franceantilles|h:guadeloupe| s:...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
7,martinique.Franceantilles.fr,2,d72b04b0015411ea9e050242ac120003,d64b735e-0154-11ea-b1d9-0242ac120004,2,https://www.martinique.franceantilles.fr,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:64.0) G...,NaN,False,ACHIEVED,...,2019-11-07T11:50:47.237Z,2019-11-07T11:50:48.797Z,2019-11-07T11:50:51.042Z,2019-11-07T12:39:43.153Z,2019-11-07T12:40:03.064Z,2.952022e+03,2.954267e+03,s:https|h:fr|h:franceantilles|h:martinique| s:...,NaN,bit.ly fb.me goo.gl j.mp l.facebook.com on.fb....
8,Gala.fr,3,d7784417015411ea9c0a0242ac120003,d64bd761-0154-11ea-9482-0242ac120004,2,https://www.gala.fr,Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...,NaN,False,ACHIEVED,...,2019-11-07T11:50:47.239Z,2019-11-07T11:50:49.263Z,2

In [ ]:
# get list of urls :

In [152]:
df = pd.read_csv (path + '\\Scrap_media_sites.csv')
df

,Unnamed: 0,name,url,title,facebook,twitter,youtube,snapchat,instagram,dailymotion
0,0,guadeloupe.franceantilles.fr,https://www.guadeloupe.franceantilles.fr,Toute l'actualité de la Guadeloupe en ligne - ...,https://www.facebook.com/FranceAntillesGuadelo...,https://twitter.com/FAGuadeloupe,NaN,NaN,https://www.instagram.com/franceantilles_guade...,NaN
1,1,martinique.franceantilles.fr,https://www.martinique.franceantilles.fr,Toute l'actualité de la Martinique en ligne - ...,https://www.facebook.com/pages/France-Antilles...,https://twitter.com/FAMartinique,NaN,NaN,https://www.instagram.com/franceantilles_marti...,NaN
2,2,gamekult.com,https://www.gamekult.com,Gamekult - Jeux vidéo PC et consoles: tout l’u...,https://www.facebook.com/gamekult/,https://twitter.com/Gamekult,https://www.youtube.com/user/gamekult,NaN,NaN,NaN
3,3,gentside.com,https://www.gentside.com,"Gentside, le portail masculin",https://www.facebook.com/303082119720215/,https://twitter.com/gentside,NaN,NaN,https://www.instagram.com/gentsidefr/,NaN
4,4,geo.fr,https://www.geo.fr,\n Le magazine de la photo et du vo...,https://www.facebook.com/GEOmagFrance,https://twitter.com/geofr,https://www.geo.fr/environnement/youtube-orien...,NaN,https://www.instagram.com/magazinegeo/,NaN
5,5,gala.fr,https://www.gala.fr,Stars : toute l'actu des stars - Gala,https://www.facebook.com/gala.fr,/l_actu/news_de_stars/angelique-de-koh-lanta-r...,NaN,NaN,NaN,NaN
6,6,gq.com,https://www.gq.com,"Men's Fashion, Style, Grooming, Fitness, Lifes...",https://www.facebook.com/gq/,https://twitter.com/gqmagazine/,https://www.youtube.com/user/gqvideos/,NaN,https://instagram.com/gq/,NaN
7,7,goldfm.fr,https://www.goldfm.fr,Accueil - Gold FM,https://www.facebook.com/goldfm33/,https://twitter.com/goldfm,https://www.youtube.com/user/GoldFm33,NaN,https://www.instagram.com/goldfm33/,NaN
8,8,glamourparis.com,https://www.glamourparis.com,"Magazine pop et engagé : société, mode, beauté...",https://www.facebook.com/glamourparis,https://twitter.com/glamourparis,https://www.youtube.com/channel/UCSnniCDEE-rap...,NaN,https://instagram.com/glamourparis/,http://www.dailymotion.com/GlamourParis
9,9,grazia.fr,https://www.grazia.fr,"Actualité mode, luxe, beauté, people, lifestyl...",https://facebook.com/graziafrance,https://twitter.com/grazia_fr,NaN,NaN,https://instagram.com/grazia_fr,NaN


0

In [485]:
## Liste des medias qui link Lemonde :

lemonde_n = graph.nodes.match(site_name="lemonde.fr").first()
Link_to_lemonde_r = graph.match(r_type="LINKS_TO", nodes=(None, lemonde_n)) 
for rel in Link_to_lemonde_r:
    print(rel['count_D2'], rel.nodes[0]['site_name'])

9 streetpress.com
1 sinemensuel.com
14 science-et-vie.com
3 revuesilence.net
45 revue-ballast.fr
15 politis.fr
3 parismatch.com
3 notretemps.com
208 mondialisation.ca
17 maxisciences.com
11 marieclaire.fr
18 marianne.net
18 magicmaman.com
5 madmoizelle.com
4 linternaute.com
4 letelegramme.fr
3 lesnumeriques.com
22 lesinrocks.com
6 lasardineduport.fr
19 yagg.com
121 usbeketrica.com
514 wikistrike.com
6 wikiagri.fr
79 wanted-pedo.com
1 vsd.fr
3 kezako.unisciel.fr
1 transrural-initiatives.org
4 terrevivante.org
94 tariqramadan.com
4 tv5monde.com
2 tribunedelyon.fr
17 topito.com
213 talents.fr
1 tetu.com
8 taranis.news
38 sud-ou-est.fr
11 socialter.fr
1 sciencepost.fr
13 francais.rt.com
28 stopmensonges.com
27 fr.sputniknews.com
102 fr.sott.net
31 reseauinternational.net
15 snj.fr
4 scienceinfo.fr
1 sante-et-travail.fr
7 samizdat.net
21 rue89bordeaux.com
7 ripostelaique.com
25 rfi.fr
1137 rezo.net
38 revolutionvibratoire.fr
29 regards.fr
3 fr.reuters.com
53 reporterre.net
1 reponseatout.co

In [482]:
# liste des sites que lemonde link avec le count
for r in graph.match((graph.nodes.match(site_name="lemonde.fr").first(),)):
    print(r['count_D2'], r.end_node['site_name'])

1 fr.sputniknews.com
1 francais.rt.com
130 talents.fr
1 sciencepost.fr
2 lesjours.fr
1 lcp.fr
2843 lavie.fr
2 l-echo.info
15 futura-sciences.com
2 ft.com
7 franceinter.fr
3 franceculture.fr
3 francebleu.fr
3 europe1.fr
1 dna.fr
2843 courrierinternational.com
2 closermag.fr
1 businessinsider.com
2 arte.tv
2 atlantico.fr
1 allodocteurs.fr
1 6play.fr
8 washingtonpost.com
1 vogue.fr
1 vidal.fr
3022 telerama.fr
1 tdg.ch
1 sudouest.fr
1 sciencesetavenir.fr
1 rugbyrama.fr
1 rue89strasbourg.com
3 rtl.fr
2 quechoisir.org
1 publicsenat.fr
2 pourlascience.fr
1 parismatch.com
2850 nouvelobs.com
1 numerama.com
1 nicematin.com
2 nextinpact.com
2972 monde-diplomatique.fr
1 midilibre.fr
5 mediapart.fr
1 marianne.net
1 linternaute.com
9 liberation.fr
3 lexpress.fr
1 letemps.ch
2 lesechos.fr
1 lequotidiendumedecin.fr
3 lequipe.fr
4 lepoint.fr
1 lemoniteur.fr
5 lejdd.fr
1 lci.fr
1 lavoixdunord.fr
1 larepubliquedespyrenees.fr
2 lapresse.ca
1 lanouvellerepublique.fr
1 lalibre.be
4 ladepeche.fr
1 la-croix.c

In [592]:
graph.delete_all()

In [727]:
## TODO : check les sleeping times and debug


for url in df2['start_urls']:
    PushUrlInfoInNeo4j(url)
    PushRSUrlInfoInNeo4j(url)

2020-02-17 17:02:49.955281 Error 5 : dw.com_fr not in hyphe.
2020-02-17 17:03:49.911337 Error 1 - lebreviairedespatriotes.fr: no file and cannot access site. SKIPPING
2020-02-17 17:03:49.912912 Error 2 : cannot parse lebreviairedespatriotes.fr
2020-02-17 17:04:11.030975 Error 1 - lebreviairedespatriotes.fr: no file and cannot access site. SKIPPING
2020-02-17 17:04:11.030975 Error 2 : cannot parse lebreviairedespatriotes.fr
2020-02-17 17:04:11.038246 Error 7 - lebreviairedespatriotes.fr: no firstpage file. Please download first 
2020-02-17 17:04:11.039212 Error 8 - lebreviairedespatriotes.fr: cannot parse RS
2020-02-17 17:05:11.890996 Error 5 : dw.com_fr not in hyphe.
2020-02-17 17:06:10.701519 Error 1 - peupledefrance.com: no file and cannot access site. SKIPPING
2020-02-17 17:06:10.703199 Error 2 : cannot parse peupledefrance.com
2020-02-17 17:06:10.981822 Error 1 - peupledefrance.com: no file and cannot access site. SKIPPING
2020-02-17 17:06:10.995623 Error 2 : cannot parse peupledef

In [728]:
for url in df['url']:
    PushUrlInfoInNeo4j(url)
    PushRSUrlInfoInNeo4j(url)

2020-02-17 17:14:52.735610 Error 5 : ajplus.net_francais not in hyphe.
2020-02-17 17:15:23.915322 Error 5 : dw.com_fr not in hyphe.
2020-02-17 17:17:12.815343 Error 5 : dw.com_fr not in hyphe.
2020-02-17 17:17:39.670806 Error 5 : frustrationmagazine.fr not in hyphe.
2020-02-17 17:17:39.904236 Error 5 : rapportsdeforce.fr not in hyphe.
2020-02-17 17:17:40.155663 Error 5 : radioparleur.net not in hyphe.
2020-02-17 17:17:40.574313 Error 5 : qg.media not in hyphe.
2020-02-17 17:17:40.859121 Error 5 : lameute.info not in hyphe.
2020-02-17 17:17:41.575592 Error 5 : lamuledupape.com not in hyphe.
2020-02-17 17:17:41.995674 Error 5 : davduf.net not in hyphe.
2020-02-17 17:17:42.190768 Error 5 : impact-s.fr not in hyphe.
2020-02-17 17:17:42.404336 Error 5 : koimagazine.fr not in hyphe.
2020-02-17 17:17:42.804198 Error 5 : lezephyrmag.com not in hyphe.
2020-02-17 17:17:43.104210 Error 5 : chut.media not in hyphe.
2020-02-17 17:17:43.378373 Error 5 : ledoc-info.com not in hyphe.
2020-02-17 17:17:

In [729]:
for url in df['url']:
    PushHypheLinksToNeo4j(url)

2020-02-17 17:17:46.275394 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:17:46.285105 Error 13 : 1>1423 cannot be created in db
2020-02-17 17:17:46.364225 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:17:46.373002 Error 13 : 2>1423 cannot be created in db
2020-02-17 17:17:46.464229 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:17:46.480079 Error 13 : 4>396 cannot be created in db
2020-02-17 17:17:46.535456 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:17:46.542595 Error 13 : 4>1158 cannot be created in db
2020-02-17 17:17:47.255893 Error 12 : france.tv /france-2 has 0 nodes in DB
2020-02-17 17:17:47.263461 Error 13 : 5>1524 cannot be created in db
2020-02-17 17:17:47.266452 Error 12 : france24.com /fr has 0 nodes in DB
2020-02-17 17:17:47.269465 Error 13 : 5>1527 cannot be created in db
2020-02-17 17:17:47.355377 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:17:47.365564 Error 13 : 6>1366 cannot be created in db
2020-02-17 17:17:47.368555 Error 12 : fra

2020-02-17 17:18:00.025210 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:18:00.033190 Error 13 : 35>1366 cannot be created in db
2020-02-17 17:18:00.053137 Error 12 : rt.com has 0 nodes in DB
2020-02-17 17:18:00.059119 Error 13 : 35>1386 cannot be created in db
2020-02-17 17:18:00.111979 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:18:00.117475 Error 13 : 38>1158 cannot be created in db
2020-02-17 17:18:00.758762 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:18:00.765746 Error 13 : 41>1158 cannot be created in db
2020-02-17 17:18:00.812620 Error 12 : lautre.net has 0 nodes in DB
2020-02-17 17:18:00.820596 Error 13 : 41>1216 cannot be created in db
2020-02-17 17:18:00.964107 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:18:00.970305 Error 13 : 41>1366 cannot be created in db
2020-02-17 17:18:01.045170 Error 12 : france24.com /fr has 0 nodes in DB
2020-02-17 17:18:01.054518 Error 13 : 41>1527 cannot be created in db
2020-02-17 17:18:01.872396 Error 12 : vice.co

2020-02-17 17:18:14.193882 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:18:14.204741 Error 13 : 105>396 cannot be created in db
2020-02-17 17:18:14.684143 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:18:14.696307 Error 13 : 105>1395 cannot be created in db
2020-02-17 17:18:14.702292 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:18:14.709274 Error 13 : 105>1423 cannot be created in db
2020-02-17 17:18:14.844002 Error 12 : orange.fr has 0 nodes in DB
2020-02-17 17:18:14.852420 Error 13 : 109>1312 cannot be created in db
2020-02-17 17:18:14.856409 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:18:14.862395 Error 13 : 109>1423 cannot be created in db
2020-02-17 17:18:15.224069 Error 12 : centrefrance.com has 0 nodes in DB
2020-02-17 17:18:15.230967 Error 13 : 111>1009 cannot be created in db
2020-02-17 17:18:15.544089 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:18:15.554595 Error 13 : 115>1423 cannot be created in db
2020-02-17 17:18:15.953752

2020-02-17 17:18:28.895371 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:18:28.904208 Error 13 : 170>1423 cannot be created in db
2020-02-17 17:18:29.235415 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:18:29.247875 Error 13 : 172>396 cannot be created in db
2020-02-17 17:18:29.355399 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:18:29.364010 Error 13 : 172>1158 cannot be created in db
2020-02-17 17:18:29.376976 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:18:29.381939 Error 13 : 172>1366 cannot be created in db
2020-02-17 17:18:29.384910 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:18:29.388926 Error 13 : 172>1423 cannot be created in db
2020-02-17 17:18:30.295369 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:18:30.305088 Error 13 : 177>1366 cannot be created in db
2020-02-17 17:18:31.435311 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:18:31.447092 Error 13 : 182>396 cannot be created in db
2020-02-17 17:18:31.504057 Error 12 : 20min.

2020-02-17 17:18:43.743685 Error 12 : lautre.net has 0 nodes in DB
2020-02-17 17:18:43.755673 Error 13 : 234>1216 cannot be created in db
2020-02-17 17:18:43.864012 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:18:43.873275 Error 13 : 234>1366 cannot be created in db
2020-02-17 17:18:43.923870 Error 12 : monde-diplomatique.fr /mav has 0 nodes in DB
2020-02-17 17:18:43.932384 Error 13 : 234>132498 cannot be created in db
2020-02-17 17:18:44.363656 Error 12 : 20min.ch /ro has 0 nodes in DB
2020-02-17 17:18:44.371547 Error 13 : 237>878 cannot be created in db
2020-02-17 17:18:44.455202 Error 12 : france.tv /france-5 has 0 nodes in DB
2020-02-17 17:18:44.464466 Error 13 : 237>1526 cannot be created in db
2020-02-17 17:18:44.523625 Error 12 : vice.com /fr has 0 nodes in DB
2020-02-17 17:18:44.538196 Error 13 : 238>132516 cannot be created in db
2020-02-17 17:18:44.615367 Error 12 : france.tv /france-2 has 0 nodes in DB
2020-02-17 17:18:44.620417 Error 13 : 239>1524 cannot be create

...

2020-02-17 17:20:49.915603 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:20:49.918884 Error 13 : 132510>1366 cannot be created in db
2020-02-17 17:20:49.935434 Error 12 : sott.net has 0 nodes in DB
2020-02-17 17:20:49.950447 Error 13 : 132510>1382 cannot be created in db
2020-02-17 17:20:49.951444 Error 12 : sputniknews.com has 0 nodes in DB
2020-02-17 17:20:49.955433 Error 13 : 132510>1383 cannot be created in db
2020-02-17 17:20:49.965405 Error 12 : rt.com has 0 nodes in DB
2020-02-17 17:20:49.969397 Error 13 : 132510>1386 cannot be created in db
2020-02-17 17:20:49.979368 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:20:49.982360 Error 13 : 132510>1395 cannot be created in db
2020-02-17 17:20:49.996409 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:20:50.012483 Error 13 : 132510>1423 cannot be created in db
2020-02-17 17:20:50.014475 Error 12 : france24.com /fr has 0 nodes in DB
2020-02-17 17:20:50.019463 Error 13 : 132510>1527 cannot be created in db

2020-02-17 17:20:56.855538 Error 12 : orange.fr has 0 nodes in DB
2020-02-17 17:20:56.874589 Error 13 : 1407>1312 cannot be created in db
2020-02-17 17:20:57.015249 Error 12 : unisciel.fr has 0 nodes in DB
2020-02-17 17:20:57.028788 Error 13 : 132515>1409 cannot be created in db
2020-02-17 17:20:57.195564 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:20:57.201656 Error 13 : 1413>1366 cannot be created in db
2020-02-17 17:20:57.289625 Error 12 : konbini.com /fr has 0 nodes in DB
2020-02-17 17:20:57.293686 Error 13 : 1414>52 cannot be created in db
2020-02-17 17:20:58.195456 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:20:58.200858 Error 13 : 1414>396 cannot be created in db
2020-02-17 17:20:58.271293 Error 12 : 20min.ch /ro has 0 nodes in DB
2020-02-17 17:20:58.276509 Error 13 : 1414>878 cannot be created in db
2020-02-17 17:20:58.755347 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:20:58.765916 Error 13 : 1414>1158 cannot be created in db
2020-02-17 17:20:58.864175 

2020-02-17 17:21:03.584415 Error 13 : 1529>1168 cannot be created in db
2020-02-17 17:21:03.615443 Error 12 : grouperf.com has 0 nodes in DB
2020-02-17 17:21:03.631258 Error 13 : 1530>1168 cannot be created in db
2020-02-17 17:21:03.675415 Error 12 : grouperf.com has 0 nodes in DB
2020-02-17 17:21:03.679631 Error 13 : 1531>1168 cannot be created in db
2020-02-17 17:21:03.715283 Error 11 : lerepublicainduzes.fr_fr has no connection in Hyphe.
2020-02-17 17:21:04.170952 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:21:04.174847 Error 13 : 173>396 cannot be created in db
2020-02-17 17:21:04.235216 Error 12 : lautre.net has 0 nodes in DB
2020-02-17 17:21:04.244004 Error 13 : 173>1216 cannot be created in db
2020-02-17 17:21:04.275668 Error 12 : france.tv /france-5 has 0 nodes in DB
2020-02-17 17:21:04.286639 Error 13 : 173>1526 cannot be created in db
2020-02-17 17:21:04.935412 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:21:04.953116 Error 13 : 183>1158 cannot be created in db
2

2020-02-17 17:21:12.595648 Error 12 : lautre.net has 0 nodes in DB
2020-02-17 17:21:12.609996 Error 13 : 323>1216 cannot be created in db
2020-02-17 17:21:12.915359 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:21:12.920809 Error 13 : 342>1366 cannot be created in db
2020-02-17 17:21:12.932777 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:21:12.937362 Error 13 : 342>1395 cannot be created in db
2020-02-17 17:21:12.991121 Error 12 : orange.fr has 0 nodes in DB
2020-02-17 17:21:12.999543 Error 13 : 347>1312 cannot be created in db
2020-02-17 17:21:13.002535 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:21:13.006524 Error 13 : 347>1366 cannot be created in db
2020-02-17 17:21:13.008519 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:21:13.012507 Error 13 : 347>1423 cannot be created in db
2020-02-17 17:21:13.335671 Error 12 : konbini.com /fr has 0 nodes in DB
2020-02-17 17:21:13.350679 Error 13 : 356>52 cannot be created in db
2020-02-17 17:21:13.61

2020-02-17 17:21:20.835565 Error 12 : consensus911.org has 0 nodes in DB
2020-02-17 17:21:20.851344 Error 13 : 132510>1033 cannot be created in db
2020-02-17 17:21:21.035370 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:21:21.043128 Error 13 : 132510>1158 cannot be created in db
2020-02-17 17:21:21.175603 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:21:21.191006 Error 13 : 132510>1366 cannot be created in db
2020-02-17 17:21:21.215506 Error 12 : sott.net has 0 nodes in DB
2020-02-17 17:21:21.231458 Error 13 : 132510>1382 cannot be created in db
2020-02-17 17:21:21.235449 Error 12 : sputniknews.com has 0 nodes in DB
2020-02-17 17:21:21.240434 Error 13 : 132510>1383 cannot be created in db
2020-02-17 17:21:21.253399 Error 12 : rt.com has 0 nodes in DB
2020-02-17 17:21:21.258536 Error 13 : 132510>1386 cannot be created in db
2020-02-17 17:21:21.272532 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:21:21.278171 Error 13 : 132510>1395 cannot be created in db
20

In [730]:
for url in df2['start_urls']:
    PushHypheLinksToNeo4j(url)


2020-02-17 17:21:23.235416 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:21:23.253969 Error 13 : 173>396 cannot be created in db
2020-02-17 17:21:23.330815 Error 12 : lautre.net has 0 nodes in DB
2020-02-17 17:21:23.338630 Error 13 : 173>1216 cannot be created in db
2020-02-17 17:21:23.390882 Error 12 : france.tv /france-5 has 0 nodes in DB
2020-02-17 17:21:23.394801 Error 13 : 173>1526 cannot be created in db
2020-02-17 17:21:24.035448 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:21:24.039437 Error 13 : 177>1366 cannot be created in db
2020-02-17 17:21:24.295264 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:21:24.300200 Error 13 : 1>1423 cannot be created in db
2020-02-17 17:21:24.335464 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:21:24.347608 Error 13 : 2>1423 cannot be created in db
2020-02-17 17:21:24.735442 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:21:24.739072 Error 13 : 4>396 cannot be created in db
2020-02-17 17:21:24.775201 Error 12 : 

2020-02-17 17:21:34.935584 Error 12 : dw.com has 0 nodes in DB
2020-02-17 17:21:34.948295 Error 13 : 34>1080 cannot be created in db
2020-02-17 17:21:35.015588 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:21:35.024190 Error 13 : 34>1395 cannot be created in db
2020-02-17 17:21:35.025188 Error 12 : theconversation.com /fr has 0 nodes in DB
2020-02-17 17:21:35.029177 Error 13 : 34>132514 cannot be created in db
2020-02-17 17:21:35.275216 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:21:35.282261 Error 13 : 35>1366 cannot be created in db
2020-02-17 17:21:35.293231 Error 12 : rt.com has 0 nodes in DB
2020-02-17 17:21:35.298184 Error 13 : 35>1386 cannot be created in db
2020-02-17 17:21:35.375585 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:21:35.392710 Error 13 : 38>1158 cannot be created in db
2020-02-17 17:21:36.055508 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:21:36.060387 Error 13 : 41>1158 cannot be created in db
2020-02-17 17:21:36.095480 Error

2020-02-17 17:21:46.756033 Error 12 : lavoixdunord.fr /sports has 0 nodes in DB
2020-02-17 17:21:46.792440 Error 13 : 103>104 cannot be created in db
2020-02-17 17:21:47.736153 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:21:47.753804 Error 13 : 103>1158 cannot be created in db
2020-02-17 17:21:47.820987 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:21:47.835785 Error 13 : 103>1423 cannot be created in db
2020-02-17 17:21:47.839743 Error 12 : france.tv /france-3 has 0 nodes in DB
2020-02-17 17:21:47.845760 Error 13 : 103>1525 cannot be created in db
2020-02-17 17:21:48.665852 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:21:48.671592 Error 13 : 105>396 cannot be created in db
2020-02-17 17:21:49.026163 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:21:49.033312 Error 13 : 105>1395 cannot be created in db
2020-02-17 17:21:49.036012 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:21:49.044981 Error 13 : 105>1423 cannot be created in db
2020-02-17 17

2020-02-17 17:22:03.455528 Error 12 : rt.com has 0 nodes in DB
2020-02-17 17:22:03.471052 Error 13 : 168>1386 cannot be created in db
2020-02-17 17:22:03.474045 Error 12 : france24.com /fr has 0 nodes in DB
2020-02-17 17:22:03.481025 Error 13 : 168>1527 cannot be created in db
2020-02-17 17:22:03.855483 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:22:03.869452 Error 13 : 170>1423 cannot be created in db
2020-02-17 17:22:04.215181 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:22:04.234597 Error 13 : 172>396 cannot be created in db
2020-02-17 17:22:04.355318 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:22:04.366954 Error 13 : 172>1158 cannot be created in db
2020-02-17 17:22:04.379921 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:22:04.385440 Error 13 : 172>1366 cannot be created in db
2020-02-17 17:22:04.388432 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:22:04.394414 Error 13 : 172>1423 cannot be created in db
2020-02-17 17:22:04.711153 Error 12 : vi

2020-02-17 17:22:19.315657 Error 12 : 20min.ch /ro has 0 nodes in DB
2020-02-17 17:22:19.327889 Error 13 : 1119>878 cannot be created in db
2020-02-17 17:22:19.575218 Error 12 : orange.fr has 0 nodes in DB
2020-02-17 17:22:19.582413 Error 13 : 1119>1312 cannot be created in db
2020-02-17 17:22:19.596465 Error 12 : rt.com has 0 nodes in DB
2020-02-17 17:22:19.615588 Error 13 : 1119>1386 cannot be created in db
2020-02-17 17:22:19.619577 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:22:19.626559 Error 13 : 1119>1395 cannot be created in db
2020-02-17 17:22:19.640566 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:22:19.658244 Error 13 : 1119>1423 cannot be created in db
2020-02-17 17:22:19.661237 Error 12 : france.tv /france-2 has 0 nodes in DB
2020-02-17 17:22:19.665258 Error 13 : 1119>1524 cannot be created in db
2020-02-17 17:22:19.668217 Error 12 : france.tv /france-5 has 0 nodes in DB
2020-02-17 17:22:19.672205 Error 13 : 1119>1526 cannot be created in db
2020-

...

2020-02-17 17:25:31.255380 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:25:31.272717 Error 13 : 182>396 cannot be created in db
2020-02-17 17:25:31.312251 Error 12 : 20min.ch /ro has 0 nodes in DB
2020-02-17 17:25:31.316299 Error 13 : 182>878 cannot be created in db
2020-02-17 17:25:31.515661 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:25:31.521370 Error 13 : 182>1158 cannot be created in db
2020-02-17 17:25:31.535363 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:25:31.545902 Error 13 : 182>1366 cannot be created in db
2020-02-17 17:25:31.556883 Error 12 : rt.com has 0 nodes in DB
2020-02-17 17:25:31.560436 Error 13 : 182>1386 cannot be created in db
2020-02-17 17:25:31.562432 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:25:31.566424 Error 13 : 182>1395 cannot be created in db
2020-02-17 17:25:31.577366 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:25:31.580680 Error 13 : 182>1423 cannot be created in db
2020-02-17 17:25:31.855370 Error 12

2020-02-17 17:25:40.935613 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:25:40.944380 Error 13 : 225>1158 cannot be created in db
2020-02-17 17:25:40.972998 Error 12 : orange.fr has 0 nodes in DB
2020-02-17 17:25:40.980118 Error 13 : 225>1312 cannot be created in db
2020-02-17 17:25:40.983111 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:25:40.987101 Error 13 : 225>1395 cannot be created in db
2020-02-17 17:25:40.998070 Error 12 : theconversation.com /fr has 0 nodes in DB
2020-02-17 17:25:41.002085 Error 13 : 225>132514 cannot be created in db
2020-02-17 17:25:42.115322 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:25:42.124919 Error 13 : 226>396 cannot be created in db
2020-02-17 17:25:42.171198 Error 12 : 20min.ch /ro has 0 nodes in DB
2020-02-17 17:25:42.176521 Error 13 : 226>878 cannot be created in db
2020-02-17 17:25:42.515495 Error 12 : ek.la has 0 nodes in DB
2020-02-17 17:25:42.533066 Error 13 : 226>1090 cannot be created in db
2020-02-17 17:25:42.65

2020-02-17 17:25:49.535512 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:25:49.553233 Error 13 : 259>396 cannot be created in db
2020-02-17 17:25:49.575255 Error 12 : 20min.ch /ro has 0 nodes in DB
2020-02-17 17:25:49.593736 Error 13 : 259>878 cannot be created in db
2020-02-17 17:25:49.695393 Error 12 : dw.com has 0 nodes in DB
2020-02-17 17:25:49.707376 Error 13 : 259>1080 cannot be created in db
2020-02-17 17:25:49.775271 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:25:49.793256 Error 13 : 259>1158 cannot be created in db
2020-02-17 17:25:49.835514 Error 12 : orange.fr has 0 nodes in DB
2020-02-17 17:25:49.845710 Error 13 : 259>1312 cannot be created in db
2020-02-17 17:25:49.859773 Error 12 : reuters.com has 0 nodes in DB
2020-02-17 17:25:49.866954 Error 13 : 259>1366 cannot be created in db
2020-02-17 17:25:49.881025 Error 12 : sputniknews.com has 0 nodes in DB
2020-02-17 17:25:49.888106 Error 13 : 259>1383 cannot be created in db
2020-02-17 17:25:49.890114 Error 12 : r

2020-02-17 17:25:56.495359 Error 12 : france24.com /fr has 0 nodes in DB
2020-02-17 17:25:56.508636 Error 13 : 273>1527 cannot be created in db
2020-02-17 17:25:56.810986 Error 12 : lautre.net has 0 nodes in DB
2020-02-17 17:25:56.817360 Error 13 : 274>1216 cannot be created in db
2020-02-17 17:25:56.831322 Error 12 : orange.fr has 0 nodes in DB
2020-02-17 17:25:56.835377 Error 13 : 274>1312 cannot be created in db
2020-02-17 17:25:56.895236 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:25:56.904853 Error 13 : 274>1395 cannot be created in db
2020-02-17 17:25:56.934446 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:25:56.939566 Error 13 : 274>1423 cannot be created in db
2020-02-17 17:25:56.941561 Error 12 : france.tv /france-5 has 0 nodes in DB
2020-02-17 17:25:56.945587 Error 13 : 274>1526 cannot be created in db
2020-02-17 17:25:57.135474 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:25:57.141355 Error 13 : 275>1395 cannot be created in db
202

2020-02-17 17:26:06.191265 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:26:06.198984 Error 13 : 327>1423 cannot be created in db
2020-02-17 17:26:06.200978 Error 12 : france.tv /france-2 has 0 nodes in DB
2020-02-17 17:26:06.203970 Error 13 : 327>1524 cannot be created in db
2020-02-17 17:26:06.796478 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:26:06.810298 Error 13 : 329>1158 cannot be created in db
2020-02-17 17:26:06.824339 Error 12 : theconversation.com has 0 nodes in DB
2020-02-17 17:26:06.834410 Error 13 : 329>1395 cannot be created in db
2020-02-17 17:26:06.837403 Error 12 : yahoo.com has 0 nodes in DB
2020-02-17 17:26:06.841365 Error 13 : 329>1423 cannot be created in db
2020-02-17 17:26:07.251260 Error 12 : vice.com has 0 nodes in DB
2020-02-17 17:26:07.255495 Error 13 : 330>396 cannot be created in db
2020-02-17 17:26:07.395672 Error 12 : ina.fr has 0 nodes in DB
2020-02-17 17:26:07.413007 Error 13 : 330>1158 cannot be created in db
2020-02-17 17:26:07.475354 Er

In [890]:
for url in df['url']:
    ImportTwitterFollowInNeo4j(url)

2020-02-18 15:58:52.353998 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 15:58:52.653539 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 15:58:53.096197 Error 14 : https://www.gala.fr has no twitter node in db
2020-02-18 15:58:53.336457 Error 15 : my_little_paris has no twitter node in db
2020-02-18 15:58:53.708018 Error 15 : my_little_paris has no twitter node in db
2020-02-18 15:58:53.926118 Error 14 : https://www.hautegironde.fr has no twitter node in db
2020-02-18 15:58:53.974988 Error 15 : futurasciences has no twitter node in db
2020-02-18 15:58:54.014965 Error 14 : https://www.hautesaintonge.fr has no twitter node in db
2020-02-18 15:58:54.123675 Error 15 : wikileaks has no twitter node in db
2020-02-18 15:58:54.473173 Error 14 : https://www.histoires-paranormales.fr has no twitter node in db
2020-02-18 15:58:54.711276 Error 15 : wikileaks has no twitter node in db
2020-02-18 15:58:55.123225 Error 15 : my_little_paris has no twitter node in db
2020-02

2020-02-18 15:59:09.887935 Error 15 : micheljanva has no twitter node in db
2020-02-18 15:59:09.965728 Error 14 : https://www.lerevenu.com has no twitter node in db
2020-02-18 15:59:09.979690 Error 14 : https://www.lesalonbeige.fr has no twitter node in db
2020-02-18 15:59:09.986670 Error 14 : https://www.leschasseursdemploi.com has no twitter node in db
2020-02-18 15:59:10.256948 Error 17 : mutinspangee has no twecoll info
2020-02-18 15:59:10.264928 Error 14 : https://www.lesoir.be has no twitter node in db
2020-02-18 15:59:10.410539 Error 15 : le_moniteur has no twitter node in db
2020-02-18 15:59:10.563363 Error 15 : le_moniteur has no twitter node in db
2020-02-18 15:59:11.009160 Error 14 : https://www.levif.be has no twitter node in db
2020-02-18 15:59:11.044064 Error 14 : https://www.lexpress.fr has no twitter node in db
2020-02-18 15:59:11.071992 Error 15 : politis_fr has no twitter node in db
2020-02-18 15:59:11.317336 Error 15 : futurasciences has no twitter node in db
2020-02

2020-02-18 15:59:26.325296 Error 15 : futurasciences has no twitter node in db
2020-02-18 15:59:26.428757 Error 16 : sninnovation has no twecoll info
2020-02-18 15:59:26.579128 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 15:59:26.622985 Error 14 : https://www.saxifrage.fr has no twitter node in db
2020-02-18 15:59:26.825171 Error 15 : journaldunet has no twitter node in db
2020-02-18 15:59:27.026987 Error 14 : https://www.sicavonline.fr has no twitter node in db
2020-02-18 15:59:27.034303 Error 14 : https://www.skyrock.com has no twitter node in db
2020-02-18 15:59:27.134183 Error 14 : https://www.soundlightup.com has no twitter node in db
2020-02-18 15:59:27.158434 Error 14 : https://www.spiil.org/s/ has no twitter node in db
2020-02-18 15:59:27.186536 Error 15 : largus_auto has no twitter node in db
2020-02-18 15:59:27.474165 Error 15 : rtbf has no twitter node in db
2020-02-18 15:59:27.807535 Error 15 : melty_fr has no twitter node in db
2020-02-18 15:59:27.815698 E

2020-02-18 15:59:38.518524 Error 15 : melty_fr has no twitter node in db
2020-02-18 15:59:38.848065 Error 14 : https://borislelay.com has no twitter node in db
2020-02-18 15:59:39.327861 Error 14 : https://www.bibamagazine.fr has no twitter node in db
2020-02-18 15:59:39.495454 Error 14 : https://breizatao.com has no twitter node in db
2020-02-18 15:59:39.501437 Error 14 : http://bretons.bzh has no twitter node in db
2020-02-18 15:59:39.504429 Error 14 : https://www.brief.me has no twitter node in db
2020-02-18 15:59:39.518472 Error 16 : bourgognemagoff has no twecoll info
2020-02-18 15:59:39.527184 Error 14 : https://www.bulletindespalion.fr has no twitter node in db
2020-02-18 15:59:39.535164 Error 14 : https://www.brochuresenligne.com has no twitter node in db
2020-02-18 15:59:39.542144 Error 16 : effluvia2 has no twecoll info
2020-02-18 15:59:39.723721 Error 15 : le_moniteur has no twitter node in db
2020-02-18 15:59:39.762741 Error 14 : http://www.buzzfil.me has no twitter node in

2020-02-18 15:59:50.832190 Error 14 : https://gerardmerinfo.fr has no twitter node in db
2020-02-18 15:59:50.842476 Error 16 : hbrfrance has no twecoll info
2020-02-18 15:59:50.848265 Error 16 : hebdoarmor has no twecoll info
2020-02-18 15:59:50.851258 Error 14 : https://herria.eus has no twitter node in db
2020-02-18 15:59:50.975300 Error 14 : http://ilfattoquotidiano.fr has no twitter node in db
2020-02-18 15:59:51.393880 Error 15 : wikileaks has no twitter node in db
2020-02-18 15:59:51.645013 Error 17 : infocontreinfo has no twecoll info
2020-02-18 15:59:51.663605 Error 16 : share has no twecoll info
2020-02-18 15:59:51.717134 Error 15 : politis_fr has no twitter node in db
2020-02-18 15:59:51.720127 Error 15 : panamza has no twitter node in db
2020-02-18 15:59:51.742066 Error 15 : wikileaks has no twitter node in db
2020-02-18 15:59:51.886807 Error 15 : micheljanva has no twitter node in db
2020-02-18 15:59:51.946353 Error 14 : https://iparraldekohitza.eus has no twitter node in d

2020-02-18 16:00:01.450453 Error 16 : metatvfrance has no twecoll info
2020-02-18 16:00:01.458598 Error 14 : http://www.minute-hebdo.fr has no twitter node in db
2020-02-18 16:00:01.531064 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:00:01.612783 Error 16 : nonspolitique has no twecoll info
2020-02-18 16:00:01.628001 Error 14 : http://www.nordactu.fr has no twitter node in db
2020-02-18 16:00:01.642523 Error 14 : http://www.nordeclair.fr has no twitter node in db
2020-02-18 16:00:01.661583 Error 14 : https://lesmoutonsrebelles.com has no twitter node in db
2020-02-18 16:00:01.667954 Error 14 : https://nordpresse.be has no twitter node in db
2020-02-18 16:00:01.678923 Error 14 : http://www.noutous.fr has no twitter node in db
2020-02-18 16:00:01.695458 Error 14 : http://www.nouveaujourj.fr has no twitter node in db
2020-02-18 16:00:02.386162 Error 14 : https://actu.orange.fr has no twitter node in db
2020-02-18 16:00:02.442737 Error 15 : politis_fr has no twitter node in

2020-02-18 16:00:11.025665 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:00:11.357845 Error 14 : https://www.letelegramme.fr has no twitter node in db
2020-02-18 16:00:11.466335 Error 14 : https://www.loirefootball.fr has no twitter node in db
2020-02-18 16:00:11.482295 Error 14 : https://www.madmoizelle.com has no twitter node in db
2020-02-18 16:00:11.611147 Error 15 : my_little_paris has no twitter node in db
2020-02-18 16:00:11.667996 Error 15 : melty_fr has no twitter node in db
2020-02-18 16:00:11.770733 Error 14 : https://www.mangerbouger.fr has no twitter node in db
2020-02-18 16:00:11.919067 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:00:12.248904 Error 15 : futurasciences has no twitter node in db
2020-02-18 16:00:12.269848 Error 14 : https://www.marchesonline.com has no twitter node in db
2020-02-18 16:00:12.382679 Error 14 : https://www.mylittleparis.com has no twitter node in db
2020-02-18 16:00:12.828944 Error 15 : journaldunet has no tw

2020-02-18 16:00:25.837079 Error 15 : rtbf has no twitter node in db
2020-02-18 16:00:25.872017 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:00:26.017634 Error 15 : politis_fr has no twitter node in db
2020-02-18 16:00:26.127024 Error 14 : https://www.fiminabeaute.com has no twitter node in db
2020-02-18 16:00:26.194972 Error 14 : http://www.ladechedumidi.com has no twitter node in db
2020-02-18 16:00:26.201159 Error 14 : http://www.ladecroissance.net has no twitter node in db
2020-02-18 16:00:26.204153 Error 14 : http://www.laminedinfos.fr has no twitter node in db
2020-02-18 16:00:26.208142 Error 14 : https://lapieuvredumidi.com has no twitter node in db
2020-02-18 16:00:26.219112 Error 14 : http://www.lavenir-hebdo.fr has no twitter node in db
2020-02-18 16:00:26.225097 Error 14 : http://www.lenouvelliste.fr has no twitter node in db
2020-02-18 16:00:26.231080 Error 14 : http://www.leconfolentais.fr has no twitter node in db
2020-02-18 16:00:26.236067 Error 14 : h

In [891]:
for url in df2['start_urls']:
    ImportTwitterFollowInNeo4j(url)

2020-02-18 16:00:29.850622 Error 17 : mutinspangee has no twecoll info
2020-02-18 16:00:29.860771 Error 14 : https://www.lesoir.be has no twitter node in db
2020-02-18 16:00:29.987422 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:00:30.089141 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:00:30.464021 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:00:30.627877 Error 14 : https://www.gala.fr has no twitter node in db
2020-02-18 16:00:31.368460 Error 15 : my_little_paris has no twitter node in db
2020-02-18 16:00:31.877641 Error 15 : my_little_paris has no twitter node in db
2020-02-18 16:00:32.212471 Error 15 : futurasciences has no twitter node in db
2020-02-18 16:00:32.271373 Error 14 : https://www.hautegironde.fr has no twitter node in db
2020-02-18 16:00:32.285368 Error 14 : https://www.hautesaintonge.fr has no twitter node in db
2020-02-18 16:00:32.355956 Error 14 : https://www.hebdo-des-savoie.com has no twitter node in db
2020-02-1

2020-02-18 16:00:47.820115 Error 14 : https://www.lepoher.fr has no twitter node in db
2020-02-18 16:00:47.946799 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:00:48.217285 Error 15 : rtbf has no twitter node in db
2020-02-18 16:00:48.537997 Error 14 : https://www.lepostillon.org has no twitter node in db
2020-02-18 16:00:48.776149 Error 14 : https://www.lequipe.fr has no twitter node in db
2020-02-18 16:00:48.780140 Error 14 : https://www.lequotidiendumedecin.fr has no twitter node in db
2020-02-18 16:00:48.784128 Error 14 : https://www.lequotidiendupharmacien.fr has no twitter node in db
2020-02-18 16:00:49.128519 Error 15 : micheljanva has no twitter node in db
2020-02-18 16:00:49.187173 Error 14 : https://www.lerevenu.com has no twitter node in db
2020-02-18 16:00:49.201363 Error 14 : https://www.lesalonbeige.fr has no twitter node in db
2020-02-18 16:00:49.208713 Error 14 : https://www.leschasseursdemploi.com has no twitter node in db
2020-02-18 16:00:49.529506 E

2020-02-18 16:01:00.039779 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:01:00.218018 Error 16 : europages has no twecoll info
2020-02-18 16:01:00.227995 Error 15 : rtbf has no twitter node in db
2020-02-18 16:01:00.374358 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:01:00.526176 Error 15 : _lebonbon has no twitter node in db
2020-02-18 16:01:00.596047 Error 15 : melty_fr has no twitter node in db
2020-02-18 16:01:00.620941 Error 15 : my_little_paris has no twitter node in db
2020-02-18 16:01:00.869033 Error 15 : journaldunet has no twitter node in db
2020-02-18 16:01:01.005926 Error 14 : https://expansive.info has no twitter node in db
2020-02-18 16:01:01.439766 Error 15 : rtbf has no twitter node in db
2020-02-18 16:01:01.480682 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:01:01.626310 Error 15 : politis_fr has no twitter node in db
2020-02-18 16:01:01.752427 Error 15 : micheljanva has no twitter node in db
2020-02-18 16:01:01.8531

2020-02-18 16:01:10.047846 Error 16 : lepelicansxm has no twecoll info
2020-02-18 16:01:10.267341 Error 16 : lepique has no twecoll info
2020-02-18 16:01:10.281362 Error 14 : https://lepressoir-info.org has no twitter node in db
2020-02-18 16:01:10.438685 Error 15 : politis_fr has no twitter node in db
2020-02-18 16:01:10.508497 Error 14 : http://www.leregional.fr has no twitter node in db
2020-02-18 16:01:10.511489 Error 14 : http://www.lerepublicainduzes.fr has no twitter node in db
2020-02-18 16:01:10.515479 Error 14 : http://www.lesaintaffricain.fr has no twitter node in db
2020-02-18 16:01:10.532436 Error 14 : https://lesakerfrancophone.fr has no twitter node in db
2020-02-18 16:01:10.536425 Error 14 : http://www.lesansculotte85.com has no twitter node in db
2020-02-18 16:01:10.802495 Error 15 : magazine_fragil has no twitter node in db
2020-02-18 16:01:10.871432 Error 14 : https://lesavoirperdudesanciens.com has no twitter node in db
2020-02-18 16:01:10.885833 Error 14 : https://

2020-02-18 16:01:19.370583 Error 14 : https://fr.sott.net has no twitter node in db
2020-02-18 16:01:19.482194 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:01:19.489213 Error 14 : https://reseaumutu.info has no twitter node in db
2020-02-18 16:01:19.530308 Error 14 : https://sciencepost.fr has no twitter node in db
2020-02-18 16:01:19.871214 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:01:20.240140 Error 14 : http://sphr.fr/hebdo/lhebdo-du-finistere/ has no twitter node in db
2020-02-18 16:01:20.247568 Error 16 : sud_ou_est has no twecoll info
2020-02-18 16:01:20.252585 Error 14 : http://swagactu.com has no twitter node in db
2020-02-18 16:01:20.269605 Error 17 : tetumag has no twecoll info
2020-02-18 16:01:20.354570 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:01:20.517951 Error 14 : http://www.talents.fr has no twitter node in db
2020-02-18 16:01:20.655313 Error 15 : melty_fr has no twitter node in db
2020-02-18 16:01:20.762311 Er

...

2020-02-18 16:02:09.650750 Error 15 : journaldunet has no twitter node in db
2020-02-18 16:02:09.757183 Error 17 : dreuz_1fo has no twecoll info
2020-02-18 16:02:10.041599 Error 15 : panamza has no twitter node in db
2020-02-18 16:02:10.061578 Error 14 : http://www.doctissimo.fr has no twitter node in db
2020-02-18 16:02:10.086477 Error 14 : https://dystopeek.fr has no twitter node in db
2020-02-18 16:02:10.092462 Error 16 : emarchespublics has no twecoll info
2020-02-18 16:02:10.096452 Error 14 : https://www.echoduberry.fr has no twitter node in db
2020-02-18 16:02:10.099444 Error 14 : http://e-penser.com has no twitter node in db
2020-02-18 16:02:10.103448 Error 14 : https://www.echoidf.fr has no twitter node in db
2020-02-18 16:02:10.107423 Error 14 : http://www.e-tribune.fr has no twitter node in db
2020-02-18 16:02:10.294204 Error 15 : le_moniteur has no twitter node in db
2020-02-18 16:02:10.479544 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:10.682158 Error

2020-02-18 16:02:26.780484 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:02:26.960469 Error 16 : europages has no twecoll info
2020-02-18 16:02:26.970339 Error 15 : rtbf has no twitter node in db
2020-02-18 16:02:27.108851 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:27.258291 Error 15 : _lebonbon has no twitter node in db
2020-02-18 16:02:27.326040 Error 15 : melty_fr has no twitter node in db
2020-02-18 16:02:27.352039 Error 15 : my_little_paris has no twitter node in db
2020-02-18 16:02:27.598231 Error 15 : journaldunet has no twitter node in db
2020-02-18 16:02:27.721267 Error 14 : https://expansive.info has no twitter node in db
2020-02-18 16:02:28.161994 Error 15 : rtbf has no twitter node in db
2020-02-18 16:02:28.195903 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:28.328653 Error 15 : politis_fr has no twitter node in db
2020-02-18 16:02:28.452673 Error 15 : micheljanva has no twitter node in db
2020-02-18 16:02:28.5501

2020-02-18 16:02:39.183671 Error 16 : espritscience has no twecoll info
2020-02-18 16:02:39.186660 Error 14 : https://www.essorsarladais.com has no twitter node in db
2020-02-18 16:02:39.512761 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:02:39.687988 Error 16 : europages has no twecoll info
2020-02-18 16:02:39.697965 Error 15 : rtbf has no twitter node in db
2020-02-18 16:02:39.822822 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:39.965458 Error 15 : _lebonbon has no twitter node in db
2020-02-18 16:02:40.037299 Error 15 : melty_fr has no twitter node in db
2020-02-18 16:02:40.062490 Error 15 : my_little_paris has no twitter node in db
2020-02-18 16:02:40.295142 Error 15 : journaldunet has no twitter node in db
2020-02-18 16:02:40.429750 Error 14 : https://expansive.info has no twitter node in db
2020-02-18 16:02:40.837123 Error 15 : rtbf has no twitter node in db
2020-02-18 16:02:40.870367 Error 15 : atlantico_fr has no twitter node in db
2020-02-

2020-02-18 16:02:54.401031 Error 14 : https://www.provenceazur-tv.fr has no twitter node in db
2020-02-18 16:02:54.492319 Error 14 : https://www.presse-pontissalienne.fr has no twitter node in db
2020-02-18 16:02:54.702237 Error 15 : politis_fr has no twitter node in db
2020-02-18 16:02:54.758842 Error 15 : futurasciences has no twitter node in db
2020-02-18 16:02:55.056416 Error 15 : magazine_fragil has no twitter node in db
2020-02-18 16:02:55.125981 Error 15 : journaldunet has no twitter node in db
2020-02-18 16:02:55.493840 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:55.907185 Error 15 : wikileaks has no twitter node in db
2020-02-18 16:02:56.448915 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:56.837515 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:56.964408 Error 15 : journaldunet has no twitter node in db
2020-02-18 16:02:57.303525 Error 15 : atlantico_fr has no twitter node in db
2020-02-18 16:02:57.311508 Error 1

2020-02-18 16:03:10.458039 Error 15 : cdlm53 has no twitter node in db
2020-02-18 16:03:10.550376 Error 14 : http://agri71.fr has no twitter node in db
2020-02-18 16:03:10.738858 Error 14 : https://www.agriculteur-charentais.fr has no twitter node in db
2020-02-18 16:03:10.838938 Error 14 : http://www.agriculteur-normand.com has no twitter node in db
2020-02-18 16:03:10.843923 Error 14 : https://www.agriculture-dromoise.fr has no twitter node in db
2020-02-18 16:03:10.933081 Error 14 : http://agriculteur-aisne.fr has no twitter node in db
2020-02-18 16:03:11.002748 Error 15 : rtbf has no twitter node in db
2020-02-18 16:03:11.017675 Error 15 : futurasciences has no twitter node in db
2020-02-18 16:03:11.257420 Error 15 : my_little_paris has no twitter node in db
2020-02-18 16:03:11.338856 Error 14 : http://www.allier-agricole.com has no twitter node in db
2020-02-18 16:03:11.651024 Error 14 : https://www.alterinfo.net has no twitter node in db
2020-02-18 16:03:11.857224 Error 15 : wiki

In [1081]:
# Pour tous les nodes twitter, download et push profile info dans neo4j

In [228]:
len(graph.nodes.match("Twitter"))

547

In [1213]:
for nodes in graph.nodes.match("Twitter"):
    try:
        push_twitpro_in_Neo4j(nodes['user'])
    except:
        print(nodes['user'])

2020-02-20 15:18:24.969666 Error 18 : hn_editions is not a twitter user
2020-02-20 15:18:25.396434 Error 18 : essorsavoyard is not a twitter user
2020-02-20 15:18:25.982846 Error 18 : parstodayfrench is not a twitter user
2020-02-20 15:18:26.284270 Error 18 : semaine03 is not a twitter user
2020-02-20 15:18:26.774914 Error 18 : 724697081890390018 is not a twitter user
2020-02-20 15:18:27.462442 Error 18 : effluvia2 is not a twitter user
2020-02-20 15:18:28.270073 Error 18 : parantsmagazine is not a twitter user
2020-02-20 15:18:28.642871 Error 18 : tepatriote is not a twitter user
2020-02-20 15:18:29.525621 Error 18 : tebeotv is not a twitter user
2020-02-20 15:18:29.842046 Error 18 : share is not a twitter user
2020-02-20 15:18:30.269013 Error 18 : obsgender is not a twitter user
2020-02-20 15:18:30.731775 Error 18 : fl24n is not a twitter user


In [1218]:
# TODO :
# - extraire les @ des descriptions twitters pour lien
for node in graph.nodes.match("Twitter"):
    try:
        if "@" in node['description']:
            print(node['description'])
    except:
        pass
    


Le portail référence de l'#immobilier professionnel - news, interviews, analyses, études, annuaire... #ininteriors, Club @Innov_Immo, conférences, formations...
Ecoutez GOLD sur 103.3 à Bordeaux, sur votre iPhone avec l'application @lesindésradios ou en streaming sur GOLDFM.FR !
🗞️ Chaque mercredi, une question d'#actualité, plusieurs regards. Un hebdomadaire papier, fondé par @ericfotto, @lgreilsamer et Nathalie Thiriez.
Le fil officiel de Gulli ! Attention, le compte @gulli_officiel n'est plus utilisé
Blogueuse @leblogdenestor 〰️ Créatrice de la Box du Grand Paris 〰️ #GrandParis #Artisanat #ProduitsLocaux
Le Peuple breton est un mensuel d'opinion créé en 1964. Rédac chef: @GaelBriand #BZH #bzhg #politique  Boutique: https://t.co/yI3Jeevp9a
Toute l'actu tourangelle en un clic. Ecrivez-nous : contact@info-tours.fr
Bienvenue ! Suivez-nous pour ne rien rater de l’actualité boursière ! #zonebourse @ZonebourseNews @ZB_Alertes
Journaliste et Producteur - Global Media - Pescivegetarien - Ret

In [ ]:
# TODO : check & correct les erreurs 5 
# TODO : blacklist les errors 1